# First Version:

## All the window sets go through the reuse at frist → Compare the profit → Give suggestion for each window sets(reuse or recycle)

In [ ]:
import random
import math
from collections import defaultdict  # Import defaultdict for convenience

random.seed(42)

# Define Netherlands geographical range (assumed)
NETHERLANDS_X_RANGE = (0, 300)  # x-coordinate range (kilometers)
NETHERLANDS_Y_RANGE = (0, 300)  # y-coordinate range (kilometers)

# Define parameters
TRANSPORT_COST_PER_KM_PER_WINDOW = 0.05   # Transport cost per km per window (EUR)
REMANUFACTURE_COST_PER_WINDOW = 18        # Remanufacture cost per window (EUR) - reduced
REUSE_SALE_PRICE_FACTOR = 150             # Base price factor for reuse (increased)
RECYCLE_SALE_PRICE_FACTOR = 80            # Base price factor for recycle (decreased)
REMANUFACTURING_DURATION = 5              # Days required for remanufacturing
DAILY_STORAGE_COST_PER_WINDOW = 2         # Daily storage cost per window (EUR)
RECYCLING_TRANSPORT_COST_MULTIPLIER = 1.2 # Increase recycling transport cost




# Define Warehouse class
class Warehouse:
    def __init__(self, name, max_storage):
        self.name = name
        self.x = random.uniform(*NETHERLANDS_X_RANGE)
        self.y = random.uniform(*NETHERLANDS_Y_RANGE)
        self.max_storage = max_storage  # Set max storage based on input
        self.daily_cost_per_window = random.uniform(1, 5)  # Daily storage cost per window (EUR)
        self.current_storage = defaultdict(int)  # Storage per day

    def __str__(self):
        return (f"{self.name}: Location=({self.x:.2f}, {self.y:.2f}), "
                f"Max Storage={self.max_storage}, Daily Cost per Window={self.daily_cost_per_window:.2f} EUR")

# Define Remanufacture Facility class
class RemanufactureFacility:
    def __init__(self):
        self.x = random.uniform(*NETHERLANDS_X_RANGE)
        self.y = random.uniform(*NETHERLANDS_Y_RANGE)
        self.daily_capacity = random.randint(50, 100)  # Daily processing capacity
        self.current_capacity = defaultdict(int)  # Used capacity per day

    def __str__(self):
        return (f"Remanufacture Facility: Location=({self.x:.2f}, {self.y:.2f}), "
                f"Daily Capacity={self.daily_capacity}")

# Define Recycling Center class
class RecyclingCenter:
    def __init__(self):
        self.x = random.uniform(*NETHERLANDS_X_RANGE)
        self.y = random.uniform(*NETHERLANDS_Y_RANGE)

    def __str__(self):
        return f"Recycling Center: Location=({self.x:.2f}, {self.y:.2f})"

# Define Reuser (Buyer) class
class Reuser:
    def __init__(self):
        self.x = random.uniform(*NETHERLANDS_X_RANGE)
        self.y = random.uniform(*NETHERLANDS_Y_RANGE)
        self.delivery_date = random.randint(20, 60)  # Delivery date between day 20 and 60

    def __str__(self):
        return f"Reuser (Buyer): Location=({self.x:.2f}, {self.y:.2f}), Delivery Date=Day {self.delivery_date}"

# Initialize the Reuser before WindowSet class definition
reuser = Reuser()

# Define Window Set class
class WindowSet:
    def __init__(self, set_id):
        self.set_id = set_id
        self.num_windows = random.randint(10, 50)  # Number of windows in the set
        self.x = random.uniform(*NETHERLANDS_X_RANGE)
        self.y = random.uniform(*NETHERLANDS_Y_RANGE)
        self.dimensions = (random.uniform(0.5, 2.0),  # Length (meters)
                           random.uniform(0.5, 2.0),  # Width (meters)
                           random.uniform(0.1, 0.5))  # Height (meters)
        self.material = random.choice(['Aluminum', 'Wood'])  # Material type
        self.condition = random.choice(['Good', 'Bad'])  # Condition
        self.demolition_date = random.randint(0, 30)  # Demolition date from day 0
        self.delivery_date = reuser.delivery_date     # Use the reuser's delivery date
        self.daily_location = {}  # Daily location tracking
        self.total_transport_cost = 0  # Total transport cost
        self.total_remanufacture_cost = 0  # Total remanufacture cost
        self.total_storage_cost = 0       # Total storage cost
        self.status = 'At Original Location'
        self.current_location_x = self.x
        self.current_location_y = self.y
        self.in_warehouse = False
        self.in_remanufacture = False
        self.delivered = False
        self.demolished = False
        self.remanufacture_complete = False
        self.remanufacture_end_day = None
        self.storage_days = 0  # Total days in warehouse
        self.warehouse_name = None  # Name of the warehouse if in warehouse
        self.waiting_for_remanufacture = False  # Waiting in warehouse for remanufacturing
        self.warehouse_entry_day = None  # Day when the window set enters the warehouse
        self.cannot_store = False  # Indicates if the window set cannot be stored

    def __str__(self):
        return (f"WindowSet {self.set_id}: Num Windows={self.num_windows}, "
                f"Location=({self.x:.2f}, {self.y:.2f}), "
                f"Dimensions=({self.dimensions[0]:.2f}, {self.dimensions[1]:.2f}, {self.dimensions[2]:.2f}), "
                f"Material={self.material}, Condition={self.condition}, "
                f"Demolition Date=Day {self.demolition_date}, Delivery Date=Day {self.delivery_date}")

# Function to calculate distance between two points (kilometers)
def calculate_distance(x1, y1, x2, y2):
    return math.hypot(x2 - x1, y2 - y1)

# Function to find the nearest warehouse with enough capacity
def find_nearest_warehouse_with_capacity(x, y, warehouses, num_windows, start_day, end_day):
    warehouses_with_capacity = []
    for wh in warehouses:
        capacity_available = True
        for day in range(start_day, end_day):
            current_storage = wh.current_storage.get(day, 0)
            if current_storage + num_windows > wh.max_storage:
                capacity_available = False
                break
        if capacity_available:
            distance = calculate_distance(x, y, wh.x, wh.y)
            warehouses_with_capacity.append((distance, wh))
    if warehouses_with_capacity:
        warehouses_with_capacity.sort(key=lambda x: x[0])
        return warehouses_with_capacity[0][1], warehouses_with_capacity[0][0]
    else:
        return None, None  # No warehouse with enough capacity

# Function to calculate sale price based on window characteristics
def calculate_sale_price(ws):
    area = ws.dimensions[0] * ws.dimensions[1]  # Window area in square meters
    material_factor = 1.2 if ws.material == 'Aluminum' else 1.0
    price_per_window = area * material_factor * REUSE_SALE_PRICE_FACTOR
    total_price = price_per_window * ws.num_windows
    return total_price

# Function to calculate recycling price based on window characteristics
def calculate_recycling_price(ws):
    area = ws.dimensions[0] * ws.dimensions[1]  # Window area in square meters
    # Introduce some randomness in material factor for recycling
    material_factor = random.uniform(0.5, 1.0) if ws.material == 'Aluminum' else random.uniform(0.3, 0.8)
    price_per_window = area * material_factor * RECYCLE_SALE_PRICE_FACTOR
    total_price = price_per_window * ws.num_windows
    return total_price

# Initialize facilities
warehouse1 = Warehouse("Warehouse 1", 900)
warehouse2 = Warehouse("Warehouse 2", 1000)
warehouses = [warehouse1, warehouse2]
remanufacture_facility = RemanufactureFacility()
recycling_center = RecyclingCenter()
# reuser = Reuser()  # Already initialized before

# Output facility information
print(warehouse1)
print(warehouse2)
print(remanufacture_facility)
print(recycling_center)
print(reuser)

# Ask user for the number of window sets to generate
num_window_sets = int(input("\nPlease enter the number of Window Sets to generate: "))

# Generate window sets
window_sets = []
for i in range(1, num_window_sets + 1):
    ws = WindowSet(i)
    window_sets.append(ws)
    print(ws)

# Simulate the process for all window sets
total_days = 60  # Simulate for 60 days
for ws in window_sets:
    ws.daily_location = {}  # Reset daily location tracking

    for day in range(total_days):
        # Initialize daily capacities and storage if not already
        # No need for explicit initialization due to defaultdict

        if day < ws.demolition_date:
            ws.daily_location[day] = 'At Original Location'
            continue

        if not ws.demolished:
            ws.demolished = True
            # Demolition day actions
            if ws.condition == 'Bad':
                # Check capacity for the entire remanufacturing duration
                can_process = True
                for d in range(day, day + REMANUFACTURING_DURATION):
                    if remanufacture_facility.current_capacity[d] + ws.num_windows > remanufacture_facility.daily_capacity:
                        can_process = False
                        break

                if can_process:
                    # Schedule remanufacturing and update capacity
                    for d in range(day, day + REMANUFACTURING_DURATION):
                        remanufacture_facility.current_capacity[d] += ws.num_windows
                    # Transport to remanufacture facility
                    distance = calculate_distance(ws.current_location_x, ws.current_location_y,
                                                  remanufacture_facility.x, remanufacture_facility.y)
                    transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                    ws.total_transport_cost += transport_cost
                    ws.current_location_x, ws.current_location_y = remanufacture_facility.x, remanufacture_facility.y
                    ws.daily_location[day] = 'In Remanufacture Facility'
                    ws.in_remanufacture = True
                    ws.remanufacture_end_day = day + REMANUFACTURING_DURATION - 1
                    # Remanufacture cost
                    remanufacture_cost = REMANUFACTURE_COST_PER_WINDOW * ws.num_windows
                    ws.total_remanufacture_cost += remanufacture_cost
                else:
                    # Try to find a warehouse with enough capacity
                    # Assume maximum storage duration until total_days
                    storage_end_day = total_days
                    nearest_warehouse, distance = find_nearest_warehouse_with_capacity(
                        ws.current_location_x, ws.current_location_y, warehouses, ws.num_windows, day, storage_end_day)
                    if nearest_warehouse is not None:
                        # Transport to warehouse
                        transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                        ws.total_transport_cost += transport_cost
                        ws.current_location_x, ws.current_location_y = nearest_warehouse.x, nearest_warehouse.y
                        ws.daily_location[day] = f"In {nearest_warehouse.name}"
                        ws.in_warehouse = True
                        ws.warehouse_name = nearest_warehouse.name
                        ws.waiting_for_remanufacture = True  # Waiting for remanufacturing
                        ws.warehouse_entry_day = day
                        # Update warehouse storage for the expected storage period
                        for d in range(day, storage_end_day):
                            nearest_warehouse.current_storage[d] += ws.num_windows
                    else:
                        # No warehouse has enough capacity; decide what to do
                        ws.daily_location[day] = 'Cannot be stored in warehouse'
                        ws.cannot_store = True
                        # Optionally, consider recycling directly
                        break  # Cannot proceed further
            else:
                # Good condition
                if day < ws.delivery_date:
                    # Estimate storage duration until delivery date
                    storage_end_day = ws.delivery_date
                    nearest_warehouse, distance = find_nearest_warehouse_with_capacity(
                        ws.current_location_x, ws.current_location_y, warehouses, ws.num_windows, day, storage_end_day)
                    if nearest_warehouse is not None:
                        # Transport to warehouse
                        transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                        ws.total_transport_cost += transport_cost
                        ws.current_location_x, ws.current_location_y = nearest_warehouse.x, nearest_warehouse.y
                        ws.daily_location[day] = f"In {nearest_warehouse.name}"
                        ws.in_warehouse = True
                        ws.warehouse_name = nearest_warehouse.name
                        ws.warehouse_entry_day = day
                        # Update warehouse storage for the expected storage period
                        for d in range(day, storage_end_day):
                            nearest_warehouse.current_storage[d] += ws.num_windows
                    else:
                        # No warehouse has enough capacity
                        ws.daily_location[day] = 'Cannot be stored in warehouse'
                        ws.cannot_store = True
                        # Optionally, consider recycling directly
                        break  # Cannot proceed further
                else:
                    # Transport to reuser
                    distance = calculate_distance(ws.current_location_x, ws.current_location_y, reuser.x, reuser.y)
                    transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                    ws.total_transport_cost += transport_cost
                    ws.current_location_x, ws.current_location_y = reuser.x, reuser.y
                    ws.daily_location[day] = 'Delivered to Reuser'
                    ws.delivered = True
                    ws.in_warehouse = False
                    break  # Delivered, no further action needed
            continue  # Move to next day

        if ws.in_remanufacture:
            if day <= ws.remanufacture_end_day:
                ws.daily_location[day] = 'In Remanufacture Facility'
                # Capacity already accounted for
            else:
                ws.in_remanufacture = False
                ws.remanufacture_complete = True
                # After remanufacturing
                if day < ws.delivery_date:
                    # Estimate storage duration until delivery date
                    storage_end_day = ws.delivery_date
                    nearest_warehouse, distance = find_nearest_warehouse_with_capacity(
                        ws.current_location_x, ws.current_location_y, warehouses, ws.num_windows, day, storage_end_day)
                    if nearest_warehouse is not None:
                        # Transport to warehouse
                        transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                        ws.total_transport_cost += transport_cost
                        ws.current_location_x, ws.current_location_y = nearest_warehouse.x, nearest_warehouse.y
                        ws.daily_location[day] = f"In {nearest_warehouse.name}"
                        ws.in_warehouse = True
                        ws.warehouse_name = nearest_warehouse.name
                        ws.warehouse_entry_day = day
                        # Update warehouse storage for the expected storage period
                        for d in range(day, storage_end_day):
                            nearest_warehouse.current_storage[d] += ws.num_windows
                    else:
                        # No warehouse has enough capacity
                        ws.daily_location[day] = 'Cannot be stored in warehouse'
                        ws.cannot_store = True
                        # Optionally, consider recycling directly
                        break  # Cannot proceed further
                else:
                    # Transport to reuser
                    distance = calculate_distance(ws.current_location_x, ws.current_location_y, reuser.x, reuser.y)
                    transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                    ws.total_transport_cost += transport_cost
                    ws.current_location_x, ws.current_location_y = reuser.x, reuser.y
                    ws.daily_location[day] = 'Delivered to Reuser'
                    ws.delivered = True
                    ws.in_warehouse = False
                    break  # Delivered, no further action needed
            continue  # Move to next day

        if ws.in_warehouse:
            ws.daily_location[day] = f"In {ws.warehouse_name}"
            # Accumulate storage cost
            storage_cost = DAILY_STORAGE_COST_PER_WINDOW * ws.num_windows
            ws.total_storage_cost += storage_cost

            if ws.waiting_for_remanufacture:
                # Try to move to remanufacture facility
                can_process = True
                for d in range(day, day + REMANUFACTURING_DURATION):
                    if remanufacture_facility.current_capacity[d] + ws.num_windows > remanufacture_facility.daily_capacity:
                        can_process = False
                        break
                if can_process:
                    # Remove from warehouse and update storage
                    ws.in_warehouse = False
                    ws.waiting_for_remanufacture = False
                    warehouse_exit_day = day
                    storage_end_day = total_days  # For waiting_for_remanufacture, storage_end_day is total_days
                    # Update warehouse storage by removing windows from current day to storage_end_day
                    for d in range(warehouse_exit_day, storage_end_day):
                        for warehouse in warehouses:
                            if warehouse.name == ws.warehouse_name:
                                warehouse.current_storage[d] -= ws.num_windows
                                if warehouse.current_storage[d] < 0:
                                    warehouse.current_storage[d] = 0  # Prevent negative storage
                                break
                    ws.warehouse_name = None
                    # Schedule remanufacturing and update capacity
                    for d in range(day, day + REMANUFACTURING_DURATION):
                        remanufacture_facility.current_capacity[d] += ws.num_windows
                    # Transport to remanufacture facility
                    distance = calculate_distance(ws.current_location_x, ws.current_location_y,
                                                  remanufacture_facility.x, remanufacture_facility.y)
                    transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                    ws.total_transport_cost += transport_cost
                    ws.current_location_x, ws.current_location_y = remanufacture_facility.x, remanufacture_facility.y
                    ws.daily_location[day] = 'In Remanufacture Facility'
                    ws.in_remanufacture = True
                    ws.remanufacture_end_day = day + REMANUFACTURING_DURATION - 1
                    # Remanufacture cost
                    remanufacture_cost = REMANUFACTURE_COST_PER_WINDOW * ws.num_windows
                    ws.total_remanufacture_cost += remanufacture_cost
                    continue  # Move to next day

            if day == ws.delivery_date and not ws.waiting_for_remanufacture:
                # Transport to reuser
                distance = calculate_distance(ws.current_location_x, ws.current_location_y, reuser.x, reuser.y)
                transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                ws.total_transport_cost += transport_cost
                ws.current_location_x, ws.current_location_y = reuser.x, reuser.y
                ws.daily_location[day] = 'Delivered to Reuser'
                ws.delivered = True
                ws.in_warehouse = False
                # Update warehouse storage by removing windows from current day to storage_end_day
                warehouse_exit_day = day
                storage_end_day = ws.delivery_date
                for d in range(warehouse_exit_day, storage_end_day):
                    for warehouse in warehouses:
                        if warehouse.name == ws.warehouse_name:
                            warehouse.current_storage[d] -= ws.num_windows
                            if warehouse.current_storage[d] < 0:
                                warehouse.current_storage[d] = 0  # Prevent negative storage
                            break
                ws.warehouse_name = None
                break  # Delivered, no further action needed
            continue  # Move to next day

    # Calculate expected reuse revenue
    total_sale_price = calculate_sale_price(ws)
    total_cost = ws.total_transport_cost + ws.total_remanufacture_cost + ws.total_storage_cost
    ws.expected_reuse_revenue = total_sale_price - total_cost

    # Calculate expected recycle revenue
    # Transport from original location to recycling center
    distance = calculate_distance(ws.x, ws.y, recycling_center.x, recycling_center.y)
    recycle_transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows * RECYCLING_TRANSPORT_COST_MULTIPLIER
    total_recycling_price = calculate_recycling_price(ws)
    ws.expected_recycle_revenue = total_recycling_price - recycle_transport_cost

# Function to print daily remanufacture processing
def print_remanufacture_daily_processing():
    print("\nDaily processing amounts in Remanufacture Facility over 60 days:")
    for day in range(total_days):
        capacity_used = remanufacture_facility.current_capacity.get(day, 0)
        print(f"Day {day}: {capacity_used} windows processed")

# Call the function to print daily remanufacture processing results
print_remanufacture_daily_processing()

def print_warehouse_daily_storage(warehouse, total_days=60):
    print(f"\nDaily storage amounts in {warehouse.name} over {total_days} days:")
    for day in range(total_days):
        capacity_used = warehouse.current_storage.get(day, 0)
        print(f"Day {day}: {capacity_used} windows stored")

# Call the functions to print daily storage results for Warehouse 1 and Warehouse 2
print_warehouse_daily_storage(warehouse1)
print_warehouse_daily_storage(warehouse2)

# Function to display information for a window set
def display_window_set_info(selected_ws):
    # Output daily locations in the specified format
    print(f"\nDaily Locations for WindowSet {selected_ws.set_id}:")
    last_location = None
    start_day = None
    days = sorted(selected_ws.daily_location.keys())
    for day in days:
        location = selected_ws.daily_location[day]
        if location != last_location:
            if last_location is not None:
                print(f"Day {start_day} - Day {day - 1}: {last_location}")
            start_day = day
            last_location = location
    # Print the last period
    if last_location is not None:
        print(f"Day {start_day} - Day {day}: {last_location}")

    # Output estimated costs and revenues
    print(f"\nEstimated Costs and Revenues for WindowSet {selected_ws.set_id}:")
    print(f"Total Transport Cost (Reuse): {selected_ws.total_transport_cost:.2f} EUR")
    print(f"Total Remanufacture Cost: {selected_ws.total_remanufacture_cost:.2f} EUR")
    print(f"Total Storage Cost: {selected_ws.total_storage_cost:.2f} EUR")
    total_cost = selected_ws.total_transport_cost + selected_ws.total_remanufacture_cost + selected_ws.total_storage_cost
    print(f"Total Cost (Reuse): {total_cost:.2f} EUR")
    total_sale_price = calculate_sale_price(selected_ws)
    print(f"Total Sale Price (Reuse): {total_sale_price:.2f} EUR")
    print(f"Expected Net Revenue (Reuse): {selected_ws.expected_reuse_revenue:.2f} EUR")

    print(f"\nRecycling Option:")
    # Transport cost to recycling center
    distance = calculate_distance(selected_ws.x, selected_ws.y, recycling_center.x, recycling_center.y)
    recycle_transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * selected_ws.num_windows * RECYCLING_TRANSPORT_COST_MULTIPLIER
    total_recycling_price = calculate_recycling_price(selected_ws)
    expected_recycle_net_revenue = total_recycling_price - recycle_transport_cost
    print(f"Transport Cost to Recycling Center: {recycle_transport_cost:.2f} EUR")
    print(f"Total Recycling Price: {total_recycling_price:.2f} EUR")
    print(f"Expected Net Revenue (Recycle): {expected_recycle_net_revenue:.2f} EUR")

    # Suggestion
    if selected_ws.expected_reuse_revenue > expected_recycle_net_revenue:
        suggestion = "Reuse"
    else:
        suggestion = "Recycle"
    print(f"\nSuggestion: {suggestion}")

# Main loop to allow user to view multiple window sets
while True:
    ws_id = input("\nWhich Window Set would you like information on? Please enter the ID number (or 'exit' to quit): ")
    if ws_id.lower() == 'exit':
        break
    try:
        ws_id = int(ws_id)
    except ValueError:
        print("Please enter a valid ID number.")
        continue
    selected_ws = next((ws for ws in window_sets if ws.set_id == ws_id), None)

    if selected_ws is None:
        print("The specified Window Set was not found.")
    else:
        display_window_set_info(selected_ws)


Warehouse 1: Location=(73.47, 41.86), Max Storage=900, Daily Cost per Window=1.41 EUR
Warehouse 2: Location=(222.20, 163.61), Max Storage=1000, Daily Cost per Window=3.36 EUR
Remanufacture Facility: Location=(9.53, 28.11), Daily Capacity=64
Recycling Center: Location=(151.61, 7.96)
Reuser (Buyer): Location=(191.83, 7.50), Delivery Date=Day 37

Please enter the number of Window Sets to generate: 10
WindowSet 1: Num Windows=22, Location=(214.81, 210.40), Dimensions=(1.13, 1.17, 0.21), Material=Aluminum, Condition=Good, Demolition Date=Day 22, Delivery Date=Day 37
WindowSet 2: Num Windows=37, Location=(102.08, 46.64), Dimensions=(1.94, 1.00, 0.14), Material=Aluminum, Condition=Bad, Demolition Date=Day 27, Delivery Date=Day 37
WindowSet 3: Num Windows=32, Location=(181.12, 242.14), Dimensions=(1.59, 1.30, 0.49), Material=Wood, Condition=Good, Demolition Date=Day 17, Delivery Date=Day 37
WindowSet 4: Num Windows=28, Location=(248.82, 185.56), Dimensions=(1.79, 1.37, 0.38), Material=Aluminum

# Improved Model:

##  Iterates over all possible combinations of reuse and recycle decisions for the window sets → Choice the maximum profit as the optimal suggestion

In [ ]:
  import random
import math
from collections import defaultdict
import itertools
import copy

random.seed(42)

# Define Netherlands geographical range (assumed)
NETHERLANDS_X_RANGE = (0, 300)  # x-coordinate range (kilometers)
NETHERLANDS_Y_RANGE = (0, 300)  # y-coordinate range (kilometers)

# Define parameters
TRANSPORT_COST_PER_KM_PER_WINDOW = 0.05   # Transport cost per km per window (EUR)
REMANUFACTURE_COST_PER_WINDOW = 18        # Remanufacture cost per window (EUR)
REUSE_SALE_PRICE_FACTOR = 150             # Base price factor for reuse
RECYCLE_SALE_PRICE_FACTOR = 80            # Base price factor for recycle
REMANUFACTURING_DURATION = 5              # Days required for remanufacturing
DAILY_STORAGE_COST_PER_WINDOW = 2         # Daily storage cost per window (EUR)
RECYCLING_TRANSPORT_COST_MULTIPLIER = 1.2 # Increase recycling transport cost

# Define Warehouse class
class Warehouse:
    def __init__(self, name, max_storage):
        self.name = name
        self.x = random.uniform(*NETHERLANDS_X_RANGE)
        self.y = random.uniform(*NETHERLANDS_Y_RANGE)
        self.max_storage = max_storage  # Set max storage based on input
        self.daily_cost_per_window = random.uniform(1, 5)  # Daily storage cost per window (EUR)
        self.current_storage = defaultdict(int)  # Storage per day

    def __str__(self):
        return (f"{self.name}: Location=({self.x:.2f}, {self.y:.2f}), "
                f"Max Storage={self.max_storage}, Daily Cost per Window={self.daily_cost_per_window:.2f} EUR")

# Define Remanufacture Facility class
class RemanufactureFacility:
    def __init__(self):
        self.x = random.uniform(*NETHERLANDS_X_RANGE)
        self.y = random.uniform(*NETHERLANDS_Y_RANGE)
        self.daily_capacity = random.randint(50, 100)  # Daily processing capacity
        self.current_capacity = defaultdict(int)  # Used capacity per day

    def __str__(self):
        return (f"Remanufacture Facility: Location=({self.x:.2f}, {self.y:.2f}), "
                f"Daily Capacity={self.daily_capacity}")

# Define Recycling Center class
class RecyclingCenter:
    def __init__(self):
        self.x = random.uniform(*NETHERLANDS_X_RANGE)
        self.y = random.uniform(*NETHERLANDS_Y_RANGE)

    def __str__(self):
        return f"Recycling Center: Location=({self.x:.2f}, {self.y:.2f})"

# Define Reuser (Buyer) class
class Reuser:
    def __init__(self):
        self.x = random.uniform(*NETHERLANDS_X_RANGE)
        self.y = random.uniform(*NETHERLANDS_Y_RANGE)
        self.delivery_date = random.randint(20, 60)  # Delivery date between day 20 and 60

    def __str__(self):
        return f"Reuser (Buyer): Location=({self.x:.2f}, {self.y:.2f}), Delivery Date=Day {self.delivery_date}"

# Initialize the Reuser before WindowSet class definition
reuser = Reuser()

# Define Window Set class
class WindowSet:
    def __init__(self, set_id):
        self.set_id = set_id
        self.num_windows = random.randint(10, 50)  # Number of windows in the set
        self.x = random.uniform(*NETHERLANDS_X_RANGE)
        self.y = random.uniform(*NETHERLANDS_Y_RANGE)
        self.dimensions = (random.uniform(0.5, 2.0),  # Length (meters)
                           random.uniform(0.5, 2.0),  # Width (meters)
                           random.uniform(0.1, 0.5))  # Height (meters)
        self.material = random.choice(['Aluminum', 'Wood'])  # Material type
        self.condition = random.choice(['Good', 'Bad'])  # Condition
        self.demolition_date = random.randint(0, 30)  # Demolition date from day 0
        self.delivery_date = reuser.delivery_date     # Use the reuser's delivery date
        self.daily_location = {}  # Daily location tracking
        self.total_transport_cost = 0  # Total transport cost
        self.total_remanufacture_cost = 0  # Total remanufacture cost
        self.total_storage_cost = 0       # Total storage cost
        self.status = 'At Original Location'
        self.current_location_x = self.x
        self.current_location_y = self.y
        self.in_warehouse = False
        self.in_remanufacture = False
        self.delivered = False
        self.demolished = False
        self.remanufacture_complete = False
        self.remanufacture_end_day = None
        self.storage_days = 0  # Total days in warehouse
        self.warehouse_name = None  # Name of the warehouse if in warehouse
        self.waiting_for_remanufacture = False  # Waiting in warehouse for remanufacturing
        self.warehouse_entry_day = None  # Day when the window set enters the warehouse
        self.cannot_store = False  # Indicates if the window set cannot be stored
        self.decision = None  # Reuse or Recycle
        self.expected_reuse_revenue = 0
        self.expected_recycle_revenue = 0

    def __str__(self):
        return (f"WindowSet {self.set_id}: Num Windows={self.num_windows}, "
                f"Location=({self.x:.2f}, {self.y:.2f}), "
                f"Dimensions=({self.dimensions[0]:.2f}, {self.dimensions[1]:.2f}, {self.dimensions[2]:.2f}), "
                f"Material={self.material}, Condition={self.condition}, "
                f"Demolition Date=Day {self.demolition_date}, Delivery Date=Day {self.delivery_date}")

# Function to calculate distance between two points (kilometers)
def calculate_distance(x1, y1, x2, y2):
    return math.hypot(x2 - x1, y2 - y1)

# Function to find the nearest warehouse with enough capacity
def find_nearest_warehouse_with_capacity(x, y, warehouses, num_windows, start_day, end_day):
    warehouses_with_capacity = []
    for wh in warehouses:
        capacity_available = True
        for day in range(start_day, end_day):
            current_storage = wh.current_storage.get(day, 0)
            if current_storage + num_windows > wh.max_storage:
                capacity_available = False
                break
        if capacity_available:
            distance = calculate_distance(x, y, wh.x, wh.y)
            warehouses_with_capacity.append((distance, wh))
    if warehouses_with_capacity:
        warehouses_with_capacity.sort(key=lambda x: x[0])
        return warehouses_with_capacity[0][1], warehouses_with_capacity[0][0]
    else:
        return None, None  # No warehouse with enough capacity

# Function to calculate sale price based on window characteristics
def calculate_sale_price(ws):
    area = ws.dimensions[0] * ws.dimensions[1]  # Window area in square meters
    material_factor = 1.2 if ws.material == 'Aluminum' else 1.0
    price_per_window = area * material_factor * REUSE_SALE_PRICE_FACTOR
    total_price = price_per_window * ws.num_windows
    return total_price

# Function to calculate recycling price based on window characteristics
def calculate_recycling_price(ws):
    area = ws.dimensions[0] * ws.dimensions[1]  # Window area in square meters
    # Introduce some randomness in material factor for recycling
    material_factor = random.uniform(0.5, 1.0) if ws.material == 'Aluminum' else random.uniform(0.3, 0.8)
    price_per_window = area * material_factor * RECYCLE_SALE_PRICE_FACTOR
    total_price = price_per_window * ws.num_windows
    return total_price

# Function to simulate the process for a given combination of decisions
def simulate_process(window_sets, decisions, warehouses, remanufacture_facility, recycling_center, reuser, total_days=60):
    # Reset facilities
    for warehouse in warehouses:
        warehouse.current_storage = defaultdict(int)
    remanufacture_facility.current_capacity = defaultdict(int)

    total_profit = 0

    # For each window set, assign the decision
    for ws, decision in zip(window_sets, decisions):
        ws.decision = decision
        # Reset window set state
        ws.daily_location = {}
        ws.total_transport_cost = 0
        ws.total_remanufacture_cost = 0
        ws.total_storage_cost = 0
        ws.status = 'At Original Location'
        ws.current_location_x = ws.x
        ws.current_location_y = ws.y
        ws.in_warehouse = False
        ws.in_remanufacture = False
        ws.delivered = False
        ws.demolished = False
        ws.remanufacture_complete = False
        ws.remanufacture_end_day = None
        ws.storage_days = 0
        ws.warehouse_name = None
        ws.waiting_for_remanufacture = False
        ws.warehouse_entry_day = None
        ws.cannot_store = False

    # Simulate day by day
    for day in range(total_days):
        # Initialize daily capacities and storage if not already (handled by defaultdict)

        for ws in window_sets:
            if day < ws.demolition_date:
                ws.daily_location[day] = 'At Original Location'
                continue

            if not ws.demolished:
                ws.demolished = True
                # Demolition day actions
                if ws.decision == 'Recycle':
                    # Transport to recycling center
                    distance = calculate_distance(ws.current_location_x, ws.current_location_y,
                                                  recycling_center.x, recycling_center.y)
                    transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows * RECYCLING_TRANSPORT_COST_MULTIPLIER
                    ws.total_transport_cost += transport_cost
                    ws.current_location_x, ws.current_location_y = recycling_center.x, recycling_center.y
                    ws.daily_location[day] = 'Sent to Recycling Center'
                    ws.delivered = True  # Considered as processed
                    break  # No further action needed
                else:
                    # Reuse decision
                    if ws.condition == 'Bad':
                        # Try to process in remanufacture facility
                        can_process = True
                        for d in range(day, day + REMANUFACTURING_DURATION):
                            if remanufacture_facility.current_capacity[d] + ws.num_windows > remanufacture_facility.daily_capacity:
                                can_process = False
                                break

                        if can_process:
                            # Schedule remanufacturing and update capacity
                            for d in range(day, day + REMANUFACTURING_DURATION):
                                remanufacture_facility.current_capacity[d] += ws.num_windows
                            # Transport to remanufacture facility
                            distance = calculate_distance(ws.current_location_x, ws.current_location_y,
                                                          remanufacture_facility.x, remanufacture_facility.y)
                            transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                            ws.total_transport_cost += transport_cost
                            ws.current_location_x, ws.current_location_y = remanufacture_facility.x, remanufacture_facility.y
                            ws.daily_location[day] = 'In Remanufacture Facility'
                            ws.in_remanufacture = True
                            ws.remanufacture_end_day = day + REMANUFACTURING_DURATION - 1
                            # Remanufacture cost
                            remanufacture_cost = REMANUFACTURE_COST_PER_WINDOW * ws.num_windows
                            ws.total_remanufacture_cost += remanufacture_cost
                        else:
                            # Try to find a warehouse with enough capacity
                            # Assume maximum storage duration until total_days
                            storage_end_day = total_days
                            nearest_warehouse, distance = find_nearest_warehouse_with_capacity(
                                ws.current_location_x, ws.current_location_y, warehouses, ws.num_windows, day, storage_end_day)
                            if nearest_warehouse is not None:
                                # Transport to warehouse
                                transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                                ws.total_transport_cost += transport_cost
                                ws.current_location_x, ws.current_location_y = nearest_warehouse.x, nearest_warehouse.y
                                ws.daily_location[day] = f"In {nearest_warehouse.name}"
                                ws.in_warehouse = True
                                ws.warehouse_name = nearest_warehouse.name
                                ws.waiting_for_remanufacture = True  # Waiting for remanufacturing
                                ws.warehouse_entry_day = day
                                # Update warehouse storage for the expected storage period
                                for d in range(day, storage_end_day):
                                    nearest_warehouse.current_storage[d] += ws.num_windows
                            else:
                                # No warehouse has enough capacity; cannot proceed
                                ws.daily_location[day] = 'Cannot be stored in warehouse'
                                ws.cannot_store = True
                                break  # Cannot proceed further
                    else:
                        # Good condition
                        if day < ws.delivery_date:
                            # Estimate storage duration until delivery date
                            storage_end_day = ws.delivery_date
                            nearest_warehouse, distance = find_nearest_warehouse_with_capacity(
                                ws.current_location_x, ws.current_location_y, warehouses, ws.num_windows, day, storage_end_day)
                            if nearest_warehouse is not None:
                                # Transport to warehouse
                                transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                                ws.total_transport_cost += transport_cost
                                ws.current_location_x, ws.current_location_y = nearest_warehouse.x, nearest_warehouse.y
                                ws.daily_location[day] = f"In {nearest_warehouse.name}"
                                ws.in_warehouse = True
                                ws.warehouse_name = nearest_warehouse.name
                                ws.warehouse_entry_day = day
                                # Update warehouse storage for the expected storage period
                                for d in range(day, storage_end_day):
                                    nearest_warehouse.current_storage[d] += ws.num_windows
                            else:
                                # No warehouse has enough capacity
                                ws.daily_location[day] = 'Cannot be stored in warehouse'
                                ws.cannot_store = True
                                break  # Cannot proceed further
                        else:
                            # Transport to reuser
                            distance = calculate_distance(ws.current_location_x, ws.current_location_y, reuser.x, reuser.y)
                            transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                            ws.total_transport_cost += transport_cost
                            ws.current_location_x, ws.current_location_y = reuser.x, reuser.y
                            ws.daily_location[day] = 'Delivered to Reuser'
                            ws.delivered = True
                            ws.in_warehouse = False
                            break  # Delivered, no further action needed
                    continue  # Move to next day

            if ws.decision == 'Recycle':
                # Already processed
                continue

            if ws.in_remanufacture:
                if day <= ws.remanufacture_end_day:
                    ws.daily_location[day] = 'In Remanufacture Facility'
                    # Capacity already accounted for
                else:
                    ws.in_remanufacture = False
                    ws.remanufacture_complete = True
                    # After remanufacturing
                    if day < ws.delivery_date:
                        # Estimate storage duration until delivery date
                        storage_end_day = ws.delivery_date
                        nearest_warehouse, distance = find_nearest_warehouse_with_capacity(
                            ws.current_location_x, ws.current_location_y, warehouses, ws.num_windows, day, storage_end_day)
                        if nearest_warehouse is not None:
                            # Transport to warehouse
                            transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                            ws.total_transport_cost += transport_cost
                            ws.current_location_x, ws.current_location_y = nearest_warehouse.x, nearest_warehouse.y
                            ws.daily_location[day] = f"In {nearest_warehouse.name}"
                            ws.in_warehouse = True
                            ws.warehouse_name = nearest_warehouse.name
                            ws.warehouse_entry_day = day
                            # Update warehouse storage for the expected storage period
                            for d in range(day, storage_end_day):
                                nearest_warehouse.current_storage[d] += ws.num_windows
                        else:
                            # No warehouse has enough capacity
                            ws.daily_location[day] = 'Cannot be stored in warehouse'
                            ws.cannot_store = True
                            break  # Cannot proceed further
                    else:
                        # Transport to reuser
                        distance = calculate_distance(ws.current_location_x, ws.current_location_y, reuser.x, reuser.y)
                        transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                        ws.total_transport_cost += transport_cost
                        ws.current_location_x, ws.current_location_y = reuser.x, reuser.y
                        ws.daily_location[day] = 'Delivered to Reuser'
                        ws.delivered = True
                        ws.in_warehouse = False
                        break  # Delivered, no further action needed
                continue  # Move to next day

            if ws.in_warehouse:
                ws.daily_location[day] = f"In {ws.warehouse_name}"
                # Accumulate storage cost
                storage_cost = DAILY_STORAGE_COST_PER_WINDOW * ws.num_windows
                ws.total_storage_cost += storage_cost

                if ws.waiting_for_remanufacture:
                    # Try to move to remanufacture facility
                    can_process = True
                    for d in range(day, day + REMANUFACTURING_DURATION):
                        if remanufacture_facility.current_capacity[d] + ws.num_windows > remanufacture_facility.daily_capacity:
                            can_process = False
                            break
                    if can_process:
                        # Remove from warehouse and update storage
                        ws.in_warehouse = False
                        ws.waiting_for_remanufacture = False
                        warehouse_exit_day = day
                        storage_end_day = total_days  # For waiting_for_remanufacture, storage_end_day is total_days
                        # Update warehouse storage by removing windows from current day to storage_end_day
                        for d in range(warehouse_exit_day, storage_end_day):
                            for warehouse in warehouses:
                                if warehouse.name == ws.warehouse_name:
                                    warehouse.current_storage[d] -= ws.num_windows
                                    if warehouse.current_storage[d] < 0:
                                        warehouse.current_storage[d] = 0  # Prevent negative storage
                                    break
                        ws.warehouse_name = None
                        # Schedule remanufacturing and update capacity
                        for d in range(day, day + REMANUFACTURING_DURATION):
                            remanufacture_facility.current_capacity[d] += ws.num_windows
                        # Transport to remanufacture facility
                        distance = calculate_distance(ws.current_location_x, ws.current_location_y,
                                                      remanufacture_facility.x, remanufacture_facility.y)
                        transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                        ws.total_transport_cost += transport_cost
                        ws.current_location_x, ws.current_location_y = remanufacture_facility.x, remanufacture_facility.y
                        ws.daily_location[day] = 'In Remanufacture Facility'
                        ws.in_remanufacture = True
                        ws.remanufacture_end_day = day + REMANUFACTURING_DURATION - 1
                        # Remanufacture cost
                        remanufacture_cost = REMANUFACTURE_COST_PER_WINDOW * ws.num_windows
                        ws.total_remanufacture_cost += remanufacture_cost
                        continue  # Move to next day

                if day == ws.delivery_date and not ws.waiting_for_remanufacture:
                    # Transport to reuser
                    distance = calculate_distance(ws.current_location_x, ws.current_location_y, reuser.x, reuser.y)
                    transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                    ws.total_transport_cost += transport_cost
                    ws.current_location_x, ws.current_location_y = reuser.x, reuser.y
                    ws.daily_location[day] = 'Delivered to Reuser'
                    ws.delivered = True
                    ws.in_warehouse = False
                    # Update warehouse storage by removing windows from current day to storage_end_day
                    warehouse_exit_day = day
                    storage_end_day = ws.delivery_date
                    for d in range(warehouse_exit_day, storage_end_day):
                        for warehouse in warehouses:
                            if warehouse.name == ws.warehouse_name:
                                warehouse.current_storage[d] -= ws.num_windows
                                if warehouse.current_storage[d] < 0:
                                    warehouse.current_storage[d] = 0  # Prevent negative storage
                                break
                    ws.warehouse_name = None
                    break  # Delivered, no further action needed
                continue  # Move to next day

        # End of day loop

    # After simulation, calculate revenues and costs
    total_profit = 0
    for ws in window_sets:
        if ws.decision == 'Reuse' and not ws.cannot_store:
            total_sale_price = calculate_sale_price(ws)
            total_cost = ws.total_transport_cost + ws.total_remanufacture_cost + ws.total_storage_cost
            ws.expected_reuse_revenue = total_sale_price - total_cost
            total_profit += ws.expected_reuse_revenue
        elif ws.decision == 'Recycle':
            total_recycling_price = calculate_recycling_price(ws)
            # Transport cost to recycling center (already calculated)
            total_cost = ws.total_transport_cost
            ws.expected_recycle_revenue = total_recycling_price - total_cost
            total_profit += ws.expected_recycle_revenue
        else:
            # Could not proceed with reuse
            ws.expected_reuse_revenue = - (ws.total_transport_cost + ws.total_remanufacture_cost + ws.total_storage_cost)
            total_profit += ws.expected_reuse_revenue  # This would be negative
    return total_profit

# Main execution
def main():
    # Initialize facilities
    warehouse1 = Warehouse("Warehouse 1", 900)
    warehouse2 = Warehouse("Warehouse 2", 1000)
    warehouses = [warehouse1, warehouse2]
    remanufacture_facility = RemanufactureFacility()
    recycling_center = RecyclingCenter()
    reuser = Reuser()

    # Output facility information
    print(warehouse1)
    print(warehouse2)
    print(remanufacture_facility)
    print(recycling_center)
    print(reuser)

    # Ask user for the number of window sets to generate
    num_window_sets = int(input("\nPlease enter the number of Window Sets to generate: "))

    # Generate window sets
    window_sets = []
    for i in range(1, num_window_sets + 1):
        ws = WindowSet(i)
        window_sets.append(ws)
        print(ws)

    # Generate all possible combinations of decisions
    decision_options = ['Reuse', 'Recycle']
    all_combinations = list(itertools.product(decision_options, repeat=num_window_sets))

    max_total_profit = float('-inf')
    min_total_profit = float('inf')

    best_combination = None
    best_window_sets = None

    print(f"\nTotal combinations to evaluate: {len(all_combinations)}")

    # For progress tracking (optional)
    combination_count = 0

    for decisions in all_combinations:
        combination_count += 1
        # Deep copy the window sets and facilities to avoid interference
        window_sets_copy = copy.deepcopy(window_sets)
        warehouses_copy = [copy.deepcopy(warehouse1), copy.deepcopy(warehouse2)]
        remanufacture_facility_copy = copy.deepcopy(remanufacture_facility)

        total_profit = simulate_process(window_sets_copy, decisions, warehouses_copy, remanufacture_facility_copy, recycling_center, reuser)

        if total_profit > max_total_profit:
            max_total_profit = total_profit
            best_combination = decisions
            best_window_sets = window_sets_copy

        # Optional: Print progress
        # print(f"Evaluated combination {combination_count}/{len(all_combinations)}: Total Profit = {total_profit:.2f} EUR")

    # Output the results for the best combination
    print(f"\nOptimal Total Profit: {max_total_profit:.2f} EUR")
    print(f"Best Combination of Decisions: {best_combination}")



# For progress tracking (worst)
    combination_count = 0

    for decisions in all_combinations:
        combination_count += 1
        # Deep copy the window sets and facilities to avoid interference
        window_sets_copy = copy.deepcopy(window_sets)
        warehouses_copy = [copy.deepcopy(warehouse1), copy.deepcopy(warehouse2)]
        remanufacture_facility_copy = copy.deepcopy(remanufacture_facility)

        total_profit = simulate_process(window_sets_copy, decisions, warehouses_copy, remanufacture_facility_copy, recycling_center, reuser)

        if total_profit < min_total_profit:
            min_total_profit = total_profit
            worst_combination = decisions
            worst_window_sets = window_sets_copy

        # Optional: Print progress
        # print(f"Evaluated combination {combination_count}/{len(all_combinations)}: Total Profit = {total_profit:.2f} EUR")

    # Output the results for the best combination
    print(f"\nMinimal Total Profit: {min_total_profit:.2f} EUR")
    print(f"Best Combination of Decisions: {worst_combination}")


    # For each window set, display the information
    for ws in best_window_sets:
        print(f"\nWindow Set {ws.set_id} Decision: {ws.decision}")
        # Output daily locations in the specified format
        print(f"Daily Locations for WindowSet {ws.set_id}:")
        last_location = None
        start_day = None
        days = sorted(ws.daily_location.keys())
        for day in days:
            location = ws.daily_location[day]
            if location != last_location:
                if last_location is not None:
                    print(f"Day {start_day} - Day {day - 1}: {last_location}")
                start_day = day
                last_location = location
        # Print the last period
        if last_location is not None:
            print(f"Day {start_day} - Day {day}: {last_location}")

        # Output estimated costs and revenues
        print(f"\nEstimated Costs and Revenues for WindowSet {ws.set_id}:")
        if ws.decision == 'Reuse':
            print(f"Total Transport Cost (Reuse): {ws.total_transport_cost:.2f} EUR")
            print(f"Total Remanufacture Cost: {ws.total_remanufacture_cost:.2f} EUR")
            print(f"Total Storage Cost: {ws.total_storage_cost:.2f} EUR")
            total_cost = ws.total_transport_cost + ws.total_remanufacture_cost + ws.total_storage_cost
            print(f"Total Cost (Reuse): {total_cost:.2f} EUR")
            total_sale_price = calculate_sale_price(ws)
            print(f"Total Sale Price (Reuse): {total_sale_price:.2f} EUR")
            print(f"Expected Net Revenue (Reuse): {ws.expected_reuse_revenue:.2f} EUR")
        else:
            print(f"Total Transport Cost (Recycle): {ws.total_transport_cost:.2f} EUR")
            total_recycling_price = calculate_recycling_price(ws)
            print(f"Total Recycling Price: {total_recycling_price:.2f} EUR")
            print(f"Expected Net Revenue (Recycle): {ws.expected_recycle_revenue:.2f} EUR")

    # Output total profit
    print(f"\nOptimal Total Profit: {max_total_profit:.2f} EUR")

    # Optionally, output warehouse storage and remanufacture facility usage for the optimal combination
    # ...

if __name__ == "__main__":
    main()


Warehouse 1: Location=(73.47, 41.86), Max Storage=900, Daily Cost per Window=1.41 EUR
Warehouse 2: Location=(222.20, 163.61), Max Storage=1000, Daily Cost per Window=3.36 EUR
Remanufacture Facility: Location=(9.53, 28.11), Daily Capacity=64
Recycling Center: Location=(151.61, 7.96)
Reuser (Buyer): Location=(59.65, 194.97), Delivery Date=Day 54

Please enter the number of Window Sets to generate: 10
WindowSet 1: Num Windows=36, Location=(66.13, 176.78), Dimensions=(1.71, 0.51, 0.42), Material=Wood, Condition=Bad, Demolition Date=Day 8, Delivery Date=Day 37
WindowSet 2: Num Windows=19, Location=(64.59, 229.05), Dimensions=(0.65, 1.07, 0.24), Material=Wood, Condition=Bad, Demolition Date=Day 25, Delivery Date=Day 37
WindowSet 3: Num Windows=12, Location=(218.92, 160.87), Dimensions=(1.96, 1.07, 0.32), Material=Wood, Condition=Good, Demolition Date=Day 22, Delivery Date=Day 37
WindowSet 4: Num Windows=14, Location=(13.75, 68.37), Dimensions=(0.93, 0.62, 0.19), Material=Aluminum, Condition=

# Updated version(Closer a lil bit to reality)

In [ ]:
import random
import math
from collections import defaultdict
import itertools
import copy

random.seed(42)

# Define Netherlands geographical range (assumed)
NETHERLANDS_X_RANGE = (0, 300)  # x-coordinate range (kilometers)
NETHERLANDS_Y_RANGE = (0, 300)  # y-coordinate range (kilometers)

# Define parameters
TRANSPORT_COST_PER_KM_PER_WINDOW = 0.05   # Transport cost per km per window (EUR)
REMANUFACTURE_COST_PER_WINDOW = 18        # Remanufacture cost per window (EUR)
REUSE_SALE_PRICE_FACTOR = 150             # Base price factor for reuse
RECYCLE_SALE_PRICE_FACTOR = 80            # Base price factor for recycle
REMANUFACTURING_DURATION = 5              # Days required for remanufacturing
DAILY_STORAGE_COST_PER_WINDOW = 2         # Daily storage cost per window (EUR)
RECYCLING_TRANSPORT_COST_MULTIPLIER = 1.2 # Increase recycling transport cost
INSPECTION_COST_PER_WINDOW = 5            # Inspection cost per window (EUR)

# Define Warehouse class
class Warehouse:
    def __init__(self, name, max_storage):
        self.name = name
        self.x = random.uniform(*NETHERLANDS_X_RANGE)
        self.y = random.uniform(*NETHERLANDS_Y_RANGE)
        self.max_storage = max_storage  # Set max storage based on input
        self.daily_cost_per_window = random.uniform(1, 5)  # Daily storage cost per window (EUR)
        self.current_storage = defaultdict(int)  # Storage per day

    def __str__(self):
        return (f"{self.name}: Location=({self.x:.2f}, {self.y:.2f}), "
                f"Max Storage={self.max_storage}, Daily Cost per Window={self.daily_cost_per_window:.2f} EUR")

# Define Remanufacture Facility class
class RemanufactureFacility:
    def __init__(self):
        self.x = random.uniform(*NETHERLANDS_X_RANGE)
        self.y = random.uniform(*NETHERLANDS_Y_RANGE)
        self.daily_capacity = random.randint(50, 100)  # Daily processing capacity
        self.current_capacity = defaultdict(int)  # Used capacity per day

    def __str__(self):
        return (f"Remanufacture Facility: Location=({self.x:.2f}, {self.y:.2f}), "
                f"Daily Capacity={self.daily_capacity}")

# Define Recycling Center class
class RecyclingCenter:
    def __init__(self):
        self.x = random.uniform(*NETHERLANDS_X_RANGE)
        self.y = random.uniform(*NETHERLANDS_Y_RANGE)

    def __str__(self):
        return f"Recycling Center: Location=({self.x:.2f}, {self.y:.2f})"

# Define Reuser (Buyer) class
class Reuser:
    def __init__(self):
        self.x = random.uniform(*NETHERLANDS_X_RANGE)
        self.y = random.uniform(*NETHERLANDS_Y_RANGE)
        self.delivery_date = random.randint(20, 60)  # Delivery date between day 20 and 60

    def __str__(self):
        return f"Reuser (Buyer): Location=({self.x:.2f}, {self.y:.2f}), Delivery Date=Day {self.delivery_date}"

# Initialize the Reuser before WindowSet class definition
reuser = Reuser()

# Define Window Set class
class WindowSet:
    def __init__(self, set_id):
        self.set_id = set_id
        self.num_windows = random.randint(10, 50)  # Number of windows in the set
        self.x = random.uniform(*NETHERLANDS_X_RANGE)
        self.y = random.uniform(*NETHERLANDS_Y_RANGE)
        self.dimensions = (random.uniform(0.5, 2.0),  # Length (meters)
                           random.uniform(0.5, 2.0),  # Width (meters)
                           random.uniform(0.1, 0.5))  # Height (meters)
        self.material = random.choice(['Aluminum', 'Wood'])  # Material type
        self.condition = random.choice(['Good', 'Bad'])  # Condition
        self.demolition_date = random.randint(0, 30)  # Demolition date from day 0
        self.delivery_date = reuser.delivery_date     # Use the reuser's delivery date
        self.information_complete = random.choice([True, False])  # Information completeness
        self.age = random.randint(1, 30)  # Age in years
        self.inspection_cost = 0
        self.inspection_done = False
        self.daily_location = {}  # Daily location tracking
        self.total_transport_cost = 0  # Total transport cost
        self.total_remanufacture_cost = 0  # Total remanufacture cost
        self.total_storage_cost = 0       # Total storage cost
        self.status = 'At Original Location'
        self.current_location_x = self.x
        self.current_location_y = self.y
        self.in_warehouse = False
        self.in_remanufacture = False
        self.delivered = False
        self.demolished = False
        self.remanufacture_complete = False
        self.remanufacture_end_day = None
        self.storage_days = 0  # Total days in warehouse
        self.warehouse_name = None  # Name of the warehouse if in warehouse
        self.waiting_for_remanufacture = False  # Waiting in warehouse for remanufacturing
        self.warehouse_entry_day = None  # Day when the window set enters the warehouse
        self.cannot_store = False  # Indicates if the window set cannot be stored
        self.decision = None  # Reuse or Recycle
        self.expected_reuse_revenue = 0
        self.expected_recycle_revenue = 0

    def __str__(self):
        return (f"WindowSet {self.set_id}: Num Windows={self.num_windows}, "
                f"Location=({self.x:.2f}, {self.y:.2f}), "
                f"Dimensions=({self.dimensions[0]:.2f}, {self.dimensions[1]:.2f}, {self.dimensions[2]:.2f}), "
                f"Material={self.material}, Condition={self.condition}, "
                f"Age={self.age}, Information Complete={self.information_complete}, "
                f"Demolition Date=Day {self.demolition_date}, Delivery Date=Day {self.delivery_date}")

# Function to calculate distance between two points (kilometers)
def calculate_distance(x1, y1, x2, y2):
    return math.hypot(x2 - x1, y2 - y1)

# Function to find the nearest warehouse with enough capacity
def find_nearest_warehouse_with_capacity(x, y, warehouses, num_windows, start_day, end_day):
    warehouses_with_capacity = []
    for wh in warehouses:
        capacity_available = True
        for day in range(start_day, end_day):
            current_storage = wh.current_storage.get(day, 0)
            if current_storage + num_windows > wh.max_storage:
                capacity_available = False
                break
        if capacity_available:
            distance = calculate_distance(x, y, wh.x, wh.y)
            warehouses_with_capacity.append((distance, wh))
    if warehouses_with_capacity:
        warehouses_with_capacity.sort(key=lambda x: x[0])
        return warehouses_with_capacity[0][1], warehouses_with_capacity[0][0]
    else:
        return None, None  # No warehouse with enough capacity

# Function to calculate sale price based on window characteristics
def calculate_sale_price(ws):
    area = ws.dimensions[0] * ws.dimensions[1]  # Window area in square meters
    material_factor = 1.2 if ws.material == 'Aluminum' else 1.0
    price_per_window = area * material_factor * REUSE_SALE_PRICE_FACTOR
    total_price = price_per_window * ws.num_windows
    return total_price

# Function to calculate recycling price based on window characteristics
def calculate_recycling_price(ws):
    area = ws.dimensions[0] * ws.dimensions[1]  # Window area in square meters
    # Introduce some randomness in material factor for recycling
    material_factor = random.uniform(0.5, 1.0) if ws.material == 'Aluminum' else random.uniform(0.3, 0.8)
    price_per_window = area * material_factor * RECYCLE_SALE_PRICE_FACTOR
    total_price = price_per_window * ws.num_windows
    return total_price

# Function to simulate the process for a given combination of decisions
def simulate_process(window_sets, decisions, warehouses, remanufacture_facility, recycling_center, reuser, total_days=60):
    # Reset facilities
    for warehouse in warehouses:
        warehouse.current_storage = defaultdict(int)
    remanufacture_facility.current_capacity = defaultdict(int)

    total_profit = 0

    # For each window set, assign the decision
    for ws, decision in zip(window_sets, decisions):
        ws.decision = decision
        # Reset window set state
        ws.daily_location = {}
        ws.total_transport_cost = 0
        ws.total_remanufacture_cost = 0
        ws.total_storage_cost = 0
        ws.inspection_cost = 0
        ws.inspection_done = False
        ws.status = 'At Original Location'
        ws.current_location_x = ws.x
        ws.current_location_y = ws.y
        ws.in_warehouse = False
        ws.in_remanufacture = False
        ws.delivered = False
        ws.demolished = False
        ws.remanufacture_complete = False
        ws.remanufacture_end_day = None
        ws.storage_days = 0
        ws.warehouse_name = None
        ws.waiting_for_remanufacture = False
        ws.warehouse_entry_day = None
        ws.cannot_store = False

    # Adjust decisions based on age
    for ws in window_sets:
        if ws.age > 25:
            ws.decision = 'Recycle'

    # Simulate day by day
    for day in range(total_days):
        # Initialize daily capacities and storage if not already (handled by defaultdict)

        for ws in window_sets:
            if day < ws.demolition_date:
                ws.daily_location[day] = 'At Original Location'
                continue

            if not ws.inspection_done and not ws.information_complete:
                # Undergo inspection
                ws.daily_location[day] = 'Under Inspection'
                ws.inspection_cost = INSPECTION_COST_PER_WINDOW * ws.num_windows
                ws.inspection_done = True
                continue  # Move to next day

            if not ws.demolished:
                ws.demolished = True
                # Demolition day actions
                if ws.decision == 'Recycle':
                    # Transport to recycling center
                    distance = calculate_distance(ws.current_location_x, ws.current_location_y,
                                                  recycling_center.x, recycling_center.y)
                    transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows * RECYCLING_TRANSPORT_COST_MULTIPLIER
                    ws.total_transport_cost += transport_cost
                    ws.current_location_x, ws.current_location_y = recycling_center.x, recycling_center.y
                    ws.daily_location[day] = 'Sent to Recycling Center'
                    ws.delivered = True  # Considered as processed
                    continue  # No further action needed
                else:
                    # Reuse decision
                    if ws.condition == 'Bad':
                        # Try to process in remanufacture facility
                        can_process = True
                        for d in range(day, day + REMANUFACTURING_DURATION):
                            if remanufacture_facility.current_capacity[d] + ws.num_windows > remanufacture_facility.daily_capacity:
                                can_process = False
                                break

                        if can_process:
                            # Schedule remanufacturing and update capacity
                            for d in range(day, day + REMANUFACTURING_DURATION):
                                remanufacture_facility.current_capacity[d] += ws.num_windows
                            # Transport to remanufacture facility
                            distance = calculate_distance(ws.current_location_x, ws.current_location_y,
                                                          remanufacture_facility.x, remanufacture_facility.y)
                            transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                            ws.total_transport_cost += transport_cost
                            ws.current_location_x, ws.current_location_y = remanufacture_facility.x, remanufacture_facility.y
                            ws.daily_location[day] = 'In Remanufacture Facility'
                            ws.in_remanufacture = True
                            ws.remanufacture_end_day = day + REMANUFACTURING_DURATION - 1
                            # Remanufacture cost
                            remanufacture_cost = REMANUFACTURE_COST_PER_WINDOW * ws.num_windows
                            ws.total_remanufacture_cost += remanufacture_cost
                        else:
                            # Try to find a warehouse with enough capacity
                            # Assume maximum storage duration until total_days
                            storage_end_day = total_days
                            nearest_warehouse, distance = find_nearest_warehouse_with_capacity(
                                ws.current_location_x, ws.current_location_y, warehouses, ws.num_windows, day, storage_end_day)
                            if nearest_warehouse is not None:
                                # Transport to warehouse
                                transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                                ws.total_transport_cost += transport_cost
                                ws.current_location_x, ws.current_location_y = nearest_warehouse.x, nearest_warehouse.y
                                ws.daily_location[day] = f"In {nearest_warehouse.name}"
                                ws.in_warehouse = True
                                ws.warehouse_name = nearest_warehouse.name
                                ws.waiting_for_remanufacture = True  # Waiting for remanufacturing
                                ws.warehouse_entry_day = day
                                # Update warehouse storage for the expected storage period
                                for d in range(day, storage_end_day):
                                    nearest_warehouse.current_storage[d] += ws.num_windows
                            else:
                                # No warehouse has enough capacity; cannot proceed
                                ws.daily_location[day] = 'Cannot be stored in warehouse'
                                ws.cannot_store = True
                                continue  # Cannot proceed further
                    else:
                        # Good condition
                        if day < ws.delivery_date:
                            # Estimate storage duration until delivery date
                            storage_end_day = ws.delivery_date
                            nearest_warehouse, distance = find_nearest_warehouse_with_capacity(
                                ws.current_location_x, ws.current_location_y, warehouses, ws.num_windows, day, storage_end_day)
                            if nearest_warehouse is not None:
                                # Transport to warehouse
                                transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                                ws.total_transport_cost += transport_cost
                                ws.current_location_x, ws.current_location_y = nearest_warehouse.x, nearest_warehouse.y
                                ws.daily_location[day] = f"In {nearest_warehouse.name}"
                                ws.in_warehouse = True
                                ws.warehouse_name = nearest_warehouse.name
                                ws.warehouse_entry_day = day
                                # Update warehouse storage for the expected storage period
                                for d in range(day, storage_end_day):
                                    nearest_warehouse.current_storage[d] += ws.num_windows
                            else:
                                # No warehouse has enough capacity
                                ws.daily_location[day] = 'Cannot be stored in warehouse'
                                ws.cannot_store = True
                                continue  # Cannot proceed further
                        else:
                            # Transport to reuser
                            distance = calculate_distance(ws.current_location_x, ws.current_location_y, reuser.x, reuser.y)
                            transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                            ws.total_transport_cost += transport_cost
                            ws.current_location_x, ws.current_location_y = reuser.x, reuser.y
                            ws.daily_location[day] = 'Delivered to Reuser'
                            ws.delivered = True
                            ws.in_warehouse = False
                            continue  # Delivered, no further action needed
                    continue  # Move to next day

            if ws.decision == 'Recycle':
                # Already processed
                continue

            if ws.in_remanufacture:
                if day <= ws.remanufacture_end_day:
                    ws.daily_location[day] = 'In Remanufacture Facility'
                    # Capacity already accounted for
                else:
                    ws.in_remanufacture = False
                    ws.remanufacture_complete = True
                    # After remanufacturing
                    if day < ws.delivery_date:
                        # Estimate storage duration until delivery date
                        storage_end_day = ws.delivery_date
                        nearest_warehouse, distance = find_nearest_warehouse_with_capacity(
                            ws.current_location_x, ws.current_location_y, warehouses, ws.num_windows, day, storage_end_day)
                        if nearest_warehouse is not None:
                            # Transport to warehouse
                            transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                            ws.total_transport_cost += transport_cost
                            ws.current_location_x, ws.current_location_y = nearest_warehouse.x, nearest_warehouse.y
                            ws.daily_location[day] = f"In {nearest_warehouse.name}"
                            ws.in_warehouse = True
                            ws.warehouse_name = nearest_warehouse.name
                            ws.warehouse_entry_day = day
                            # Update warehouse storage for the expected storage period
                            for d in range(day, storage_end_day):
                                nearest_warehouse.current_storage[d] += ws.num_windows
                        else:
                            # No warehouse has enough capacity
                            ws.daily_location[day] = 'Cannot be stored in warehouse'
                            ws.cannot_store = True
                            continue  # Cannot proceed further
                    else:
                        # Transport to reuser
                        distance = calculate_distance(ws.current_location_x, ws.current_location_y, reuser.x, reuser.y)
                        transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                        ws.total_transport_cost += transport_cost
                        ws.current_location_x, ws.current_location_y = reuser.x, reuser.y
                        ws.daily_location[day] = 'Delivered to Reuser'
                        ws.delivered = True
                        ws.in_warehouse = False
                        continue  # Delivered, no further action needed
                continue  # Move to next day

            if ws.in_warehouse:
                ws.daily_location[day] = f"In {ws.warehouse_name}"
                # Accumulate storage cost
                storage_cost = DAILY_STORAGE_COST_PER_WINDOW * ws.num_windows
                ws.total_storage_cost += storage_cost

                if ws.waiting_for_remanufacture:
                    # Try to move to remanufacture facility
                    can_process = True
                    for d in range(day, day + REMANUFACTURING_DURATION):
                        if remanufacture_facility.current_capacity[d] + ws.num_windows > remanufacture_facility.daily_capacity:
                            can_process = False
                            break
                    if can_process:
                        # Remove from warehouse and update storage
                        ws.in_warehouse = False
                        ws.waiting_for_remanufacture = False
                        warehouse_exit_day = day
                        storage_end_day = total_days  # For waiting_for_remanufacture, storage_end_day is total_days
                        # Update warehouse storage by removing windows from current day to storage_end_day
                        for d in range(warehouse_exit_day, storage_end_day):
                            for warehouse in warehouses:
                                if warehouse.name == ws.warehouse_name:
                                    warehouse.current_storage[d] -= ws.num_windows
                                    if warehouse.current_storage[d] < 0:
                                        warehouse.current_storage[d] = 0  # Prevent negative storage
                                    break
                        ws.warehouse_name = None
                        # Schedule remanufacturing and update capacity
                        for d in range(day, day + REMANUFACTURING_DURATION):
                            remanufacture_facility.current_capacity[d] += ws.num_windows
                        # Transport to remanufacture facility
                        distance = calculate_distance(ws.current_location_x, ws.current_location_y,
                                                      remanufacture_facility.x, remanufacture_facility.y)
                        transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                        ws.total_transport_cost += transport_cost
                        ws.current_location_x, ws.current_location_y = remanufacture_facility.x, remanufacture_facility.y
                        ws.daily_location[day] = 'In Remanufacture Facility'
                        ws.in_remanufacture = True
                        ws.remanufacture_end_day = day + REMANUFACTURING_DURATION - 1
                        # Remanufacture cost
                        remanufacture_cost = REMANUFACTURE_COST_PER_WINDOW * ws.num_windows
                        ws.total_remanufacture_cost += remanufacture_cost
                        continue  # Move to next day

                if day == ws.delivery_date and not ws.waiting_for_remanufacture:
                    # Transport to reuser
                    distance = calculate_distance(ws.current_location_x, ws.current_location_y, reuser.x, reuser.y)
                    transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                    ws.total_transport_cost += transport_cost
                    ws.current_location_x, ws.current_location_y = reuser.x, reuser.y
                    ws.daily_location[day] = 'Delivered to Reuser'
                    ws.delivered = True
                    ws.in_warehouse = False
                    # Update warehouse storage by removing windows from current day to storage_end_day
                    warehouse_exit_day = day
                    storage_end_day = ws.delivery_date
                    for d in range(warehouse_exit_day, storage_end_day):
                        for warehouse in warehouses:
                            if warehouse.name == ws.warehouse_name:
                                warehouse.current_storage[d] -= ws.num_windows
                                if warehouse.current_storage[d] < 0:
                                    warehouse.current_storage[d] = 0  # Prevent negative storage
                                break
                    ws.warehouse_name = None
                    continue  # Delivered, no further action needed
                continue  # Move to next day

        # End of day loop

    # After simulation, calculate revenues and costs
    total_profit = 0
    for ws in window_sets:
        if ws.decision == 'Reuse' and not ws.cannot_store:
            total_sale_price = calculate_sale_price(ws)
            total_cost = ws.total_transport_cost + ws.total_remanufacture_cost + ws.total_storage_cost + ws.inspection_cost
            ws.expected_reuse_revenue = total_sale_price - total_cost
            total_profit += ws.expected_reuse_revenue
        elif ws.decision == 'Recycle':
            total_recycling_price = calculate_recycling_price(ws)
            # Transport cost to recycling center (already calculated)
            total_cost = ws.total_transport_cost + ws.inspection_cost
            ws.expected_recycle_revenue = total_recycling_price - total_cost
            total_profit += ws.expected_recycle_revenue
        else:
            # Could not proceed with reuse
            total_cost = ws.total_transport_cost + ws.total_remanufacture_cost + ws.total_storage_cost + ws.inspection_cost
            ws.expected_reuse_revenue = - total_cost
            total_profit += ws.expected_reuse_revenue  # This would be negative
    return total_profit

# Main execution
def main():
    # Initialize facilities
    warehouse1 = Warehouse("Warehouse 1", 900)
    warehouse2 = Warehouse("Warehouse 2", 1000)
    warehouses = [warehouse1, warehouse2]
    remanufacture_facility = RemanufactureFacility()
    recycling_center = RecyclingCenter()
    reuser = Reuser()

    # Output facility information
    print(warehouse1)
    print(warehouse2)
    print(remanufacture_facility)
    print(recycling_center)
    print(reuser)

    # Ask user for the number of window sets to generate
    num_window_sets = int(input("\nPlease enter the number of Window Sets to generate: "))

    # Generate window sets
    window_sets = []
    for i in range(1, num_window_sets + 1):
        ws = WindowSet(i)
        window_sets.append(ws)
        print(ws)

    # Generate decision options per window set
    decision_options_list = []
    for ws in window_sets:
        if ws.age > 25:
            decision_options_list.append(['Recycle'])
        else:
            decision_options_list.append(['Reuse', 'Recycle'])

    # Generate all possible combinations of decisions
    all_combinations = list(itertools.product(*decision_options_list))

    max_total_profit = float('-inf')
    min_total_profit = float('inf')

    best_combination = None
    best_window_sets = None

    print(f"\nTotal combinations to evaluate: {len(all_combinations)}")

    # For progress tracking (optional)
    combination_count = 0

    for decisions in all_combinations:
        combination_count += 1
        # Deep copy the window sets and facilities to avoid interference
        window_sets_copy = copy.deepcopy(window_sets)
        warehouses_copy = [copy.deepcopy(warehouse1), copy.deepcopy(warehouse2)]
        remanufacture_facility_copy = copy.deepcopy(remanufacture_facility)

        total_profit = simulate_process(window_sets_copy, decisions, warehouses_copy, remanufacture_facility_copy, recycling_center, reuser)

        if total_profit > max_total_profit:
            max_total_profit = total_profit
            best_combination = decisions
            best_window_sets = window_sets_copy

        # Optional: Print progress
        # print(f"Evaluated combination {combination_count}/{len(all_combinations)}: Total Profit = {total_profit:.2f} EUR")

    # Output the results for the best combination
    print(f"\nOptimal Total Profit: {max_total_profit:.2f} EUR")
    print(f"Best Combination of Decisions: {best_combination}")

    # For progress tracking (worst)
    combination_count = 0

    for decisions in all_combinations:
        combination_count += 1
        # Deep copy the window sets and facilities to avoid interference
        window_sets_copy = copy.deepcopy(window_sets)
        warehouses_copy = [copy.deepcopy(warehouse1), copy.deepcopy(warehouse2)]
        remanufacture_facility_copy = copy.deepcopy(remanufacture_facility)

        total_profit = simulate_process(window_sets_copy, decisions, warehouses_copy, remanufacture_facility_copy, recycling_center, reuser)

        if total_profit < min_total_profit:
            min_total_profit = total_profit
            worst_combination = decisions
            worst_window_sets = window_sets_copy

        # Optional: Print progress
        # print(f"Evaluated combination {combination_count}/{len(all_combinations)}: Total Profit = {total_profit:.2f} EUR")

    # Output the results for the worst combination
    print(f"\nMinimal Total Profit: {min_total_profit:.2f} EUR")
    print(f"Worst Combination of Decisions: {worst_combination}")

    # For each window set, display the information
    for ws in best_window_sets:
        print(f"\nWindow Set {ws.set_id} Decision: {ws.decision}")
        # Output daily locations in the specified format
        print(f"Daily Locations for WindowSet {ws.set_id}:")
        last_location = None
        start_day = None
        days = sorted(ws.daily_location.keys())
        for day in days:
            location = ws.daily_location[day]
            if location != last_location:
                if last_location is not None:
                    print(f"Day {start_day} - Day {day - 1}: {last_location}")
                start_day = day
                last_location = location
        # Print the last period
        if last_location is not None:
            print(f"Day {start_day} - Day {day}: {last_location}")

        # Output estimated costs and revenues
        print(f"\nEstimated Costs and Revenues for WindowSet {ws.set_id}:")
        if ws.decision == 'Reuse':
            print(f"Total Transport Cost (Reuse): {ws.total_transport_cost:.2f} EUR")
            print(f"Total Remanufacture Cost: {ws.total_remanufacture_cost:.2f} EUR")
            print(f"Total Storage Cost: {ws.total_storage_cost:.2f} EUR")
            print(f"Inspection Cost: {ws.inspection_cost:.2f} EUR")
            total_cost = ws.total_transport_cost + ws.total_remanufacture_cost + ws.total_storage_cost + ws.inspection_cost
            print(f"Total Cost (Reuse): {total_cost:.2f} EUR")
            total_sale_price = calculate_sale_price(ws)
            print(f"Total Sale Price (Reuse): {total_sale_price:.2f} EUR")
            print(f"Expected Net Revenue (Reuse): {ws.expected_reuse_revenue:.2f} EUR")
        else:
            print(f"Total Transport Cost (Recycle): {ws.total_transport_cost:.2f} EUR")
            print(f"Inspection Cost: {ws.inspection_cost:.2f} EUR")
            total_recycling_price = calculate_recycling_price(ws)
            print(f"Total Recycling Price: {total_recycling_price:.2f} EUR")
            print(f"Expected Net Revenue (Recycle): {ws.expected_recycle_revenue:.2f} EUR")

    # Output total profit
    print(f"\nOptimal Total Profit: {max_total_profit:.2f} EUR")

    # Optionally, output warehouse storage and remanufacture facility usage for the optimal combination
    # ...

if __name__ == "__main__":
    main()


Warehouse 1: Location=(73.47, 41.86), Max Storage=900, Daily Cost per Window=1.41 EUR
Warehouse 2: Location=(222.20, 163.61), Max Storage=1000, Daily Cost per Window=3.36 EUR
Remanufacture Facility: Location=(9.53, 28.11), Daily Capacity=64
Recycling Center: Location=(151.61, 7.96)
Reuser (Buyer): Location=(59.65, 194.97), Delivery Date=Day 54

Please enter the number of Window Sets to generate: 10
WindowSet 1: Num Windows=36, Location=(66.13, 176.78), Dimensions=(1.71, 0.51, 0.42), Material=Wood, Condition=Bad, Age=7, Information Complete=True, Demolition Date=Day 8, Delivery Date=Day 37
WindowSet 2: Num Windows=31, Location=(30.66, 113.98), Dimensions=(1.04, 1.02, 0.21), Material=Aluminum, Condition=Bad, Age=30, Information Complete=True, Demolition Date=Day 17, Delivery Date=Day 37
WindowSet 3: Num Windows=34, Location=(23.64, 87.95), Dimensions=(1.44, 1.83, 0.24), Material=Aluminum, Condition=Good, Age=25, Information Complete=True, Demolition Date=Day 1, Delivery Date=Day 37
Windo

# Add sustainability

In [ ]:
import random
import math
from collections import defaultdict
import itertools
import copy

random.seed(42)

# Define Netherlands geographical range (assumed)
NETHERLANDS_X_RANGE = (0, 300)  # x-coordinate range (kilometers)
NETHERLANDS_Y_RANGE = (0, 300)  # y-coordinate range (kilometers)

# Define parameters
TRANSPORT_COST_PER_KM_PER_WINDOW = 0.05   # Transport cost per km per window (EUR)
REMANUFACTURE_COST_PER_WINDOW = 18        # Remanufacture cost per window (EUR)
REUSE_SALE_PRICE_FACTOR = 150             # Base price factor for reuse
RECYCLE_SALE_PRICE_FACTOR = 80            # Base price factor for recycle
REMANUFACTURING_DURATION = 5              # Days required for remanufacturing
DAILY_STORAGE_COST_PER_WINDOW = 2         # Daily storage cost per window (EUR)
RECYCLING_TRANSPORT_COST_MULTIPLIER = 1.2 # Increase recycling transport cost
INSPECTION_COST_PER_WINDOW = 5            # Inspection cost per window (EUR)

# Sustainability parameters
TRANSPORT_EMISSION_FACTOR = 0.0002        # kg CO2 per km per window
REMANUFACTURING_EMISSION_PER_WINDOW = 5   # kg CO2 per window
RECYCLING_EMISSION_PER_WINDOW = 2         # kg CO2 per window
CARBON_PRICE_PER_KG = 0.05                # EUR per kg CO2

# Define Warehouse class
class Warehouse:
    def __init__(self, name, max_storage):
        self.name = name
        self.x = random.uniform(*NETHERLANDS_X_RANGE)
        self.y = random.uniform(*NETHERLANDS_Y_RANGE)
        self.max_storage = max_storage  # Set max storage based on input
        self.daily_cost_per_window = random.uniform(1, 5)  # Daily storage cost per window (EUR)
        self.current_storage = defaultdict(int)  # Storage per day

    def __str__(self):
        return (f"{self.name}: Location=({self.x:.2f}, {self.y:.2f}), "
                f"Max Storage={self.max_storage}, Daily Cost per Window={self.daily_cost_per_window:.2f} EUR")

# Define Remanufacture Facility class
class RemanufactureFacility:
    def __init__(self):
        self.x = random.uniform(*NETHERLANDS_X_RANGE)
        self.y = random.uniform(*NETHERLANDS_Y_RANGE)
        self.daily_capacity = random.randint(50, 100)  # Daily processing capacity
        self.current_capacity = defaultdict(int)  # Used capacity per day

    def __str__(self):
        return (f"Remanufacture Facility: Location=({self.x:.2f}, {self.y:.2f}), "
                f"Daily Capacity={self.daily_capacity}")

# Define Recycling Center class
class RecyclingCenter:
    def __init__(self):
        self.x = random.uniform(*NETHERLANDS_X_RANGE)
        self.y = random.uniform(*NETHERLANDS_Y_RANGE)

    def __str__(self):
        return f"Recycling Center: Location=({self.x:.2f}, {self.y:.2f})"

# Define Reuser (Buyer) class
class Reuser:
    def __init__(self):
        self.x = random.uniform(*NETHERLANDS_X_RANGE)
        self.y = random.uniform(*NETHERLANDS_Y_RANGE)
        self.delivery_date = random.randint(20, 60)  # Delivery date between day 20 and 60

    def __str__(self):
        return f"Reuser (Buyer): Location=({self.x:.2f}, {self.y:.2f}), Delivery Date=Day {self.delivery_date}"

# Initialize the Reuser before WindowSet class definition
reuser = Reuser()

# Define Window Set class
class WindowSet:
    def __init__(self, set_id):
        self.set_id = set_id
        self.num_windows = random.randint(10, 50)  # Number of windows in the set
        self.x = random.uniform(*NETHERLANDS_X_RANGE)
        self.y = random.uniform(*NETHERLANDS_Y_RANGE)
        self.dimensions = (random.uniform(0.5, 2.0),  # Length (meters)
                           random.uniform(0.5, 2.0),  # Width (meters)
                           random.uniform(0.1, 0.5))  # Height (meters)
        self.material = random.choice(['Aluminum', 'Wood'])  # Material type
        self.condition = random.choice(['Good', 'Bad'])  # Condition
        self.demolition_date = random.randint(0, 30)  # Demolition date from day 0
        self.delivery_date = reuser.delivery_date     # Use the reuser's delivery date
        self.information_complete = random.choice([True, False])  # Information completeness
        self.age = random.randint(1, 30)  # Age in years
        self.inspection_cost = 0
        self.inspection_done = False
        self.daily_location = {}  # Daily location tracking
        self.total_transport_cost = 0  # Total transport cost
        self.total_remanufacture_cost = 0  # Total remanufacture cost
        self.total_storage_cost = 0       # Total storage cost
        self.status = 'At Original Location'
        self.current_location_x = self.x
        self.current_location_y = self.y
        self.in_warehouse = False
        self.in_remanufacture = False
        self.delivered = False
        self.demolished = False
        self.remanufacture_complete = False
        self.remanufacture_end_day = None
        self.storage_days = 0  # Total days in warehouse
        self.warehouse_name = None  # Name of the warehouse if in warehouse
        self.waiting_for_remanufacture = False  # Waiting in warehouse for remanufacturing
        self.warehouse_entry_day = None  # Day when the window set enters the warehouse
        self.cannot_store = False  # Indicates if the window set cannot be stored
        self.decision = None  # Reuse or Recycle
        self.expected_reuse_revenue = 0
        self.expected_recycle_revenue = 0
        # Sustainability metrics
        self.total_transport_emissions = 0  # kg CO2
        self.total_remanufacture_emissions = 0  # kg CO2
        self.total_recycling_emissions = 0  # kg CO2
        self.resource_savings = 0  # kg of material saved
        self.total_emissions = 0  # Total emissions kg CO2
        self.carbon_cost = 0  # EUR

    def __str__(self):
        return (f"WindowSet {self.set_id}: Num Windows={self.num_windows}, "
                f"Location=({self.x:.2f}, {self.y:.2f}), "
                f"Dimensions=({self.dimensions[0]:.2f}, {self.dimensions[1]:.2f}, {self.dimensions[2]:.2f}), "
                f"Material={self.material}, Condition={self.condition}, "
                f"Age={self.age}, Information Complete={self.information_complete}, "
                f"Demolition Date=Day {self.demolition_date}, Delivery Date=Day {self.delivery_date}")

# Function to calculate distance between two points (kilometers)
def calculate_distance(x1, y1, x2, y2):
    return math.hypot(x2 - x1, y2 - y1)

# Function to find the nearest warehouse with enough capacity
def find_nearest_warehouse_with_capacity(x, y, warehouses, num_windows, start_day, end_day):
    warehouses_with_capacity = []
    for wh in warehouses:
        capacity_available = True
        for day in range(start_day, end_day):
            current_storage = wh.current_storage.get(day, 0)
            if current_storage + num_windows > wh.max_storage:
                capacity_available = False
                break
        if capacity_available:
            distance = calculate_distance(x, y, wh.x, wh.y)
            warehouses_with_capacity.append((distance, wh))
    if warehouses_with_capacity:
        warehouses_with_capacity.sort(key=lambda x: x[0])
        return warehouses_with_capacity[0][1], warehouses_with_capacity[0][0]
    else:
        return None, None  # No warehouse with enough capacity

# Function to calculate sale price based on window characteristics
def calculate_sale_price(ws):
    area = ws.dimensions[0] * ws.dimensions[1]  # Window area in square meters
    material_factor = 1.2 if ws.material == 'Aluminum' else 1.0
    price_per_window = area * material_factor * REUSE_SALE_PRICE_FACTOR
    total_price = price_per_window * ws.num_windows
    return total_price

# Function to calculate recycling price based on window characteristics
def calculate_recycling_price(ws):
    area = ws.dimensions[0] * ws.dimensions[1]  # Window area in square meters
    # Introduce some randomness in material factor for recycling
    material_factor = random.uniform(0.5, 1.0) if ws.material == 'Aluminum' else random.uniform(0.3, 0.8)
    price_per_window = area * material_factor * RECYCLE_SALE_PRICE_FACTOR
    total_price = price_per_window * ws.num_windows
    return total_price

# Function to estimate window weight (kg) based on material and dimensions
def estimate_window_weight(ws):
    density = 2700 if ws.material == 'Aluminum' else 600  # kg/m^3 (approximate densities)
    volume = ws.dimensions[0] * ws.dimensions[1] * ws.dimensions[2]  # m^3
    weight = density * volume  # kg per window
    total_weight = weight * ws.num_windows
    return total_weight

# Function to simulate the process for a given combination of decisions
def simulate_process(window_sets, decisions, warehouses, remanufacture_facility, recycling_center, reuser, total_days=60):
    # Reset facilities
    for warehouse in warehouses:
        warehouse.current_storage = defaultdict(int)
    remanufacture_facility.current_capacity = defaultdict(int)

    total_profit = 0
    total_emissions = 0  # Total emissions for all window sets (kg CO2)

    # For each window set, assign the decision
    for ws, decision in zip(window_sets, decisions):
        ws.decision = decision
        # Reset window set state
        ws.daily_location = {}
        ws.total_transport_cost = 0
        ws.total_remanufacture_cost = 0
        ws.total_storage_cost = 0
        ws.inspection_cost = 0
        ws.inspection_done = False
        ws.status = 'At Original Location'
        ws.current_location_x = ws.x
        ws.current_location_y = ws.y
        ws.in_warehouse = False
        ws.in_remanufacture = False
        ws.delivered = False
        ws.demolished = False
        ws.remanufacture_complete = False
        ws.remanufacture_end_day = None
        ws.storage_days = 0
        ws.warehouse_name = None
        ws.waiting_for_remanufacture = False
        ws.warehouse_entry_day = None
        ws.cannot_store = False
        # Reset sustainability metrics
        ws.total_transport_emissions = 0
        ws.total_remanufacture_emissions = 0
        ws.total_recycling_emissions = 0
        ws.resource_savings = 0
        ws.total_emissions = 0
        ws.carbon_cost = 0

    # Adjust decisions based on age
    for ws in window_sets:
        if ws.age > 25:
            ws.decision = 'Recycle'

    # Simulate day by day
    for day in range(total_days):
        # Initialize daily capacities and storage if not already (handled by defaultdict)

        for ws in window_sets:
            if day < ws.demolition_date:
                ws.daily_location[day] = 'At Original Location'
                continue

            if not ws.inspection_done and not ws.information_complete:
                # Undergo inspection
                ws.daily_location[day] = 'Under Inspection'
                ws.inspection_cost = INSPECTION_COST_PER_WINDOW * ws.num_windows
                ws.inspection_done = True
                continue  # Move to next day

            if not ws.demolished:
                ws.demolished = True
                # Demolition day actions
                if ws.decision == 'Recycle':
                    # Transport to recycling center
                    distance = calculate_distance(ws.current_location_x, ws.current_location_y,
                                                  recycling_center.x, recycling_center.y)
                    transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows * RECYCLING_TRANSPORT_COST_MULTIPLIER
                    ws.total_transport_cost += transport_cost
                    # Calculate transport emissions
                    transport_emissions = distance * TRANSPORT_EMISSION_FACTOR * ws.num_windows * RECYCLING_TRANSPORT_COST_MULTIPLIER
                    ws.total_transport_emissions += transport_emissions
                    ws.current_location_x, ws.current_location_y = recycling_center.x, recycling_center.y
                    ws.daily_location[day] = 'Sent to Recycling Center'
                    # Recycling emissions
                    recycling_emissions = RECYCLING_EMISSION_PER_WINDOW * ws.num_windows
                    ws.total_recycling_emissions += recycling_emissions
                    # Total emissions
                    ws.total_emissions = ws.total_transport_emissions + ws.total_recycling_emissions
                    ws.delivered = True  # Considered as processed
                    continue  # No further action needed
                else:
                    # Reuse decision
                    if ws.condition == 'Bad':
                        # Try to process in remanufacture facility
                        can_process = True
                        for d in range(day, day + REMANUFACTURING_DURATION):
                            if remanufacture_facility.current_capacity[d] + ws.num_windows > remanufacture_facility.daily_capacity:
                                can_process = False
                                break

                        if can_process:
                            # Schedule remanufacturing and update capacity
                            for d in range(day, day + REMANUFACTURING_DURATION):
                                remanufacture_facility.current_capacity[d] += ws.num_windows
                            # Transport to remanufacture facility
                            distance = calculate_distance(ws.current_location_x, ws.current_location_y,
                                                          remanufacture_facility.x, remanufacture_facility.y)
                            transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                            ws.total_transport_cost += transport_cost
                            # Calculate transport emissions
                            transport_emissions = distance * TRANSPORT_EMISSION_FACTOR * ws.num_windows
                            ws.total_transport_emissions += transport_emissions
                            ws.current_location_x, ws.current_location_y = remanufacture_facility.x, remanufacture_facility.y
                            ws.daily_location[day] = 'In Remanufacture Facility'
                            ws.in_remanufacture = True
                            ws.remanufacture_end_day = day + REMANUFACTURING_DURATION - 1
                            # Remanufacture cost
                            remanufacture_cost = REMANUFACTURE_COST_PER_WINDOW * ws.num_windows
                            ws.total_remanufacture_cost += remanufacture_cost
                            # Remanufacturing emissions
                            remanufacture_emissions = REMANUFACTURING_EMISSION_PER_WINDOW * ws.num_windows
                            ws.total_remanufacture_emissions += remanufacture_emissions
                        else:
                            # Try to find a warehouse with enough capacity
                            # Assume maximum storage duration until total_days
                            storage_end_day = total_days
                            nearest_warehouse, distance = find_nearest_warehouse_with_capacity(
                                ws.current_location_x, ws.current_location_y, warehouses, ws.num_windows, day, storage_end_day)
                            if nearest_warehouse is not None:
                                # Transport to warehouse
                                transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                                ws.total_transport_cost += transport_cost
                                # Calculate transport emissions
                                transport_emissions = distance * TRANSPORT_EMISSION_FACTOR * ws.num_windows
                                ws.total_transport_emissions += transport_emissions
                                ws.current_location_x, ws.current_location_y = nearest_warehouse.x, nearest_warehouse.y
                                ws.daily_location[day] = f"In {nearest_warehouse.name}"
                                ws.in_warehouse = True
                                ws.warehouse_name = nearest_warehouse.name
                                ws.waiting_for_remanufacture = True  # Waiting for remanufacturing
                                ws.warehouse_entry_day = day
                                # Update warehouse storage for the expected storage period
                                for d in range(day, storage_end_day):
                                    nearest_warehouse.current_storage[d] += ws.num_windows
                            else:
                                # No warehouse has enough capacity; cannot proceed
                                ws.daily_location[day] = 'Cannot be stored in warehouse'
                                ws.cannot_store = True
                                continue  # Cannot proceed further
                    else:
                        # Good condition
                        if day < ws.delivery_date:
                            # Estimate storage duration until delivery date
                            storage_end_day = ws.delivery_date
                            nearest_warehouse, distance = find_nearest_warehouse_with_capacity(
                                ws.current_location_x, ws.current_location_y, warehouses, ws.num_windows, day, storage_end_day)
                            if nearest_warehouse is not None:
                                # Transport to warehouse
                                transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                                ws.total_transport_cost += transport_cost
                                # Calculate transport emissions
                                transport_emissions = distance * TRANSPORT_EMISSION_FACTOR * ws.num_windows
                                ws.total_transport_emissions += transport_emissions
                                ws.current_location_x, ws.current_location_y = nearest_warehouse.x, nearest_warehouse.y
                                ws.daily_location[day] = f"In {nearest_warehouse.name}"
                                ws.in_warehouse = True
                                ws.warehouse_name = nearest_warehouse.name
                                ws.warehouse_entry_day = day
                                # Update warehouse storage for the expected storage period
                                for d in range(day, storage_end_day):
                                    nearest_warehouse.current_storage[d] += ws.num_windows
                            else:
                                # No warehouse has enough capacity
                                ws.daily_location[day] = 'Cannot be stored in warehouse'
                                ws.cannot_store = True
                                continue  # Cannot proceed further
                        else:
                            # Transport to reuser
                            distance = calculate_distance(ws.current_location_x, ws.current_location_y, reuser.x, reuser.y)
                            transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                            ws.total_transport_cost += transport_cost
                            # Calculate transport emissions
                            transport_emissions = distance * TRANSPORT_EMISSION_FACTOR * ws.num_windows
                            ws.total_transport_emissions += transport_emissions
                            ws.current_location_x, ws.current_location_y = reuser.x, reuser.y
                            ws.daily_location[day] = 'Delivered to Reuser'
                            ws.delivered = True
                            ws.in_warehouse = False
                            # Resource savings
                            total_weight = estimate_window_weight(ws)
                            ws.resource_savings = total_weight  # kg of material saved by reusing
                            # Total emissions (transport emissions already accounted)
                            ws.total_emissions = ws.total_transport_emissions
                            continue  # Delivered, no further action needed
                    continue  # Move to next day

            if ws.decision == 'Recycle':
                # Already processed
                continue

            if ws.in_remanufacture:
                if day <= ws.remanufacture_end_day:
                    ws.daily_location[day] = 'In Remanufacture Facility'
                    # Capacity already accounted for
                else:
                    ws.in_remanufacture = False
                    ws.remanufacture_complete = True
                    # After remanufacturing
                    if day < ws.delivery_date:
                        # Estimate storage duration until delivery date
                        storage_end_day = ws.delivery_date
                        nearest_warehouse, distance = find_nearest_warehouse_with_capacity(
                            ws.current_location_x, ws.current_location_y, warehouses, ws.num_windows, day, storage_end_day)
                        if nearest_warehouse is not None:
                            # Transport to warehouse
                            transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                            ws.total_transport_cost += transport_cost
                            # Calculate transport emissions
                            transport_emissions = distance * TRANSPORT_EMISSION_FACTOR * ws.num_windows
                            ws.total_transport_emissions += transport_emissions
                            ws.current_location_x, ws.current_location_y = nearest_warehouse.x, nearest_warehouse.y
                            ws.daily_location[day] = f"In {nearest_warehouse.name}"
                            ws.in_warehouse = True
                            ws.warehouse_name = nearest_warehouse.name
                            ws.warehouse_entry_day = day
                            # Update warehouse storage for the expected storage period
                            for d in range(day, storage_end_day):
                                nearest_warehouse.current_storage[d] += ws.num_windows
                        else:
                            # No warehouse has enough capacity
                            ws.daily_location[day] = 'Cannot be stored in warehouse'
                            ws.cannot_store = True
                            continue  # Cannot proceed further
                    else:
                        # Transport to reuser
                        distance = calculate_distance(ws.current_location_x, ws.current_location_y, reuser.x, reuser.y)
                        transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                        ws.total_transport_cost += transport_cost
                        # Calculate transport emissions
                        transport_emissions = distance * TRANSPORT_EMISSION_FACTOR * ws.num_windows
                        ws.total_transport_emissions += transport_emissions
                        ws.current_location_x, ws.current_location_y = reuser.x, reuser.y
                        ws.daily_location[day] = 'Delivered to Reuser'
                        ws.delivered = True
                        ws.in_warehouse = False
                        # Resource savings
                        total_weight = estimate_window_weight(ws)
                        ws.resource_savings = total_weight  # kg of material saved by reusing
                        # Total emissions (transport and remanufacturing emissions already accounted)
                        ws.total_emissions = ws.total_transport_emissions + ws.total_remanufacture_emissions
                        continue  # Delivered, no further action needed
                continue  # Move to next day

            if ws.in_warehouse:
                ws.daily_location[day] = f"In {ws.warehouse_name}"
                # Accumulate storage cost
                storage_cost = DAILY_STORAGE_COST_PER_WINDOW * ws.num_windows
                ws.total_storage_cost += storage_cost

                if ws.waiting_for_remanufacture:
                    # Try to move to remanufacture facility
                    can_process = True
                    for d in range(day, day + REMANUFACTURING_DURATION):
                        if remanufacture_facility.current_capacity[d] + ws.num_windows > remanufacture_facility.daily_capacity:
                            can_process = False
                            break
                    if can_process:
                        # Remove from warehouse and update storage
                        ws.in_warehouse = False
                        ws.waiting_for_remanufacture = False
                        warehouse_exit_day = day
                        storage_end_day = total_days  # For waiting_for_remanufacture, storage_end_day is total_days
                        # Update warehouse storage by removing windows from current day to storage_end_day
                        for d in range(warehouse_exit_day, storage_end_day):
                            for warehouse in warehouses:
                                if warehouse.name == ws.warehouse_name:
                                    warehouse.current_storage[d] -= ws.num_windows
                                    if warehouse.current_storage[d] < 0:
                                        warehouse.current_storage[d] = 0  # Prevent negative storage
                                    break
                        ws.warehouse_name = None
                        # Schedule remanufacturing and update capacity
                        for d in range(day, day + REMANUFACTURING_DURATION):
                            remanufacture_facility.current_capacity[d] += ws.num_windows
                        # Transport to remanufacture facility
                        distance = calculate_distance(ws.current_location_x, ws.current_location_y,
                                                      remanufacture_facility.x, remanufacture_facility.y)
                        transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                        ws.total_transport_cost += transport_cost
                        # Calculate transport emissions
                        transport_emissions = distance * TRANSPORT_EMISSION_FACTOR * ws.num_windows
                        ws.total_transport_emissions += transport_emissions
                        ws.current_location_x, ws.current_location_y = remanufacture_facility.x, remanufacture_facility.y
                        ws.daily_location[day] = 'In Remanufacture Facility'
                        ws.in_remanufacture = True
                        ws.remanufacture_end_day = day + REMANUFACTURING_DURATION - 1
                        # Remanufacture cost
                        remanufacture_cost = REMANUFACTURE_COST_PER_WINDOW * ws.num_windows
                        ws.total_remanufacture_cost += remanufacture_cost
                        # Remanufacturing emissions
                        remanufacture_emissions = REMANUFACTURING_EMISSION_PER_WINDOW * ws.num_windows
                        ws.total_remanufacture_emissions += remanufacture_emissions
                        continue  # Move to next day

                if day == ws.delivery_date and not ws.waiting_for_remanufacture:
                    # Transport to reuser
                    distance = calculate_distance(ws.current_location_x, ws.current_location_y, reuser.x, reuser.y)
                    transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                    ws.total_transport_cost += transport_cost
                    # Calculate transport emissions
                    transport_emissions = distance * TRANSPORT_EMISSION_FACTOR * ws.num_windows
                    ws.total_transport_emissions += transport_emissions
                    ws.current_location_x, ws.current_location_y = reuser.x, reuser.y
                    ws.daily_location[day] = 'Delivered to Reuser'
                    ws.delivered = True
                    ws.in_warehouse = False
                    # Update warehouse storage by removing windows from current day to storage_end_day
                    warehouse_exit_day = day
                    storage_end_day = ws.delivery_date
                    for d in range(warehouse_exit_day, storage_end_day):
                        for warehouse in warehouses:
                            if warehouse.name == ws.warehouse_name:
                                warehouse.current_storage[d] -= ws.num_windows
                                if warehouse.current_storage[d] < 0:
                                    warehouse.current_storage[d] = 0  # Prevent negative storage
                                break
                    ws.warehouse_name = None
                    # Resource savings
                    total_weight = estimate_window_weight(ws)
                    ws.resource_savings = total_weight  # kg of material saved by reusing
                    # Total emissions (transport emissions already accounted)
                    ws.total_emissions = ws.total_transport_emissions
                    continue  # Delivered, no further action needed
                continue  # Move to next day

        # End of day loop

    # After simulation, calculate revenues and costs
    total_profit = 0
    total_emissions = 0  # kg CO2
    total_resource_savings = 0  # kg of material saved

    for ws in window_sets:
        if ws.decision == 'Reuse' and not ws.cannot_store:
            total_sale_price = calculate_sale_price(ws)
            total_cost = ws.total_transport_cost + ws.total_remanufacture_cost + ws.total_storage_cost + ws.inspection_cost
            ws.expected_reuse_revenue = total_sale_price - total_cost
            total_profit += ws.expected_reuse_revenue
            # Carbon cost
            ws.carbon_cost = ws.total_emissions * CARBON_PRICE_PER_KG
            total_profit -= ws.carbon_cost
            total_emissions += ws.total_emissions
            total_resource_savings += ws.resource_savings
        elif ws.decision == 'Recycle':
            total_recycling_price = calculate_recycling_price(ws)
            # Transport cost to recycling center (already calculated)
            total_cost = ws.total_transport_cost + ws.inspection_cost
            ws.expected_recycle_revenue = total_recycling_price - total_cost
            total_profit += ws.expected_recycle_revenue
            # Carbon cost
            ws.carbon_cost = ws.total_emissions * CARBON_PRICE_PER_KG
            total_profit -= ws.carbon_cost
            total_emissions += ws.total_emissions
            # No resource savings in recycling
        else:
            # Could not proceed with reuse
            total_cost = ws.total_transport_cost + ws.total_remanufacture_cost + ws.total_storage_cost + ws.inspection_cost
            ws.expected_reuse_revenue = - total_cost
            total_profit += ws.expected_reuse_revenue  # This would be negative
            # Carbon cost
            ws.carbon_cost = ws.total_emissions * CARBON_PRICE_PER_KG
            total_profit -= ws.carbon_cost
            total_emissions += ws.total_emissions
            # No resource savings
    return total_profit, total_emissions, total_resource_savings

# Main execution
def main():
    # Initialize facilities
    warehouse1 = Warehouse("Warehouse 1", 900)
    warehouse2 = Warehouse("Warehouse 2", 1000)
    warehouses = [warehouse1, warehouse2]
    remanufacture_facility = RemanufactureFacility()
    recycling_center = RecyclingCenter()
    reuser = Reuser()

    # Output facility information
    print(warehouse1)
    print(warehouse2)
    print(remanufacture_facility)
    print(recycling_center)
    print(reuser)

    # Ask user for the number of window sets to generate
    num_window_sets = int(input("\nPlease enter the number of Window Sets to generate: "))

    # Generate window sets
    window_sets = []
    for i in range(1, num_window_sets + 1):
        ws = WindowSet(i)
        window_sets.append(ws)
        print(ws)

    # Generate decision options per window set
    decision_options_list = []
    for ws in window_sets:
        if ws.age > 25:
            decision_options_list.append(['Recycle'])
        else:
            decision_options_list.append(['Reuse', 'Recycle'])

    # Generate all possible combinations of decisions
    all_combinations = list(itertools.product(*decision_options_list))

    max_total_profit = float('-inf')
    min_total_profit = float('inf')

    best_combination = None
    best_window_sets = None
    best_total_emissions = None
    best_total_resource_savings = None

    print(f"\nTotal combinations to evaluate: {len(all_combinations)}")

    # For progress tracking (optional)
    combination_count = 0

    for decisions in all_combinations:
        combination_count += 1
        # Deep copy the window sets and facilities to avoid interference
        window_sets_copy = copy.deepcopy(window_sets)
        warehouses_copy = [copy.deepcopy(warehouse1), copy.deepcopy(warehouse2)]
        remanufacture_facility_copy = copy.deepcopy(remanufacture_facility)

        total_profit, total_emissions, total_resource_savings = simulate_process(
            window_sets_copy, decisions, warehouses_copy, remanufacture_facility_copy, recycling_center, reuser)

        if total_profit > max_total_profit:
            max_total_profit = total_profit
            best_combination = decisions
            best_window_sets = window_sets_copy
            best_total_emissions = total_emissions
            best_total_resource_savings = total_resource_savings

        # Optional: Print progress
        # print(f"Evaluated combination {combination_count}/{len(all_combinations)}: Total Profit = {total_profit:.2f} EUR")

    # Output the results for the best combination
    print(f"\nOptimal Total Profit: {max_total_profit:.2f} EUR")
    print(f"Total Emissions: {best_total_emissions:.2f} kg CO2")
    print(f"Total Resource Savings: {best_total_resource_savings:.2f} kg of material")
    print(f"Best Combination of Decisions: {best_combination}")

    # For each window set, display the information
    for ws in best_window_sets:
        print(f"\nWindow Set {ws.set_id} Decision: {ws.decision}")
        # Output daily locations in the specified format
        print(f"Daily Locations for WindowSet {ws.set_id}:")
        last_location = None
        start_day = None
        days = sorted(ws.daily_location.keys())
        for day in days:
            location = ws.daily_location[day]
            if location != last_location:
                if last_location is not None:
                    print(f"Day {start_day} - Day {day - 1}: {last_location}")
                start_day = day
                last_location = location
        # Print the last period
        if last_location is not None:
            print(f"Day {start_day} - Day {day}: {last_location}")

        # Output estimated costs and revenues
        print(f"\nEstimated Costs and Revenues for WindowSet {ws.set_id}:")
        if ws.decision == 'Reuse':
            print(f"Total Transport Cost (Reuse): {ws.total_transport_cost:.2f} EUR")
            print(f"Total Remanufacture Cost: {ws.total_remanufacture_cost:.2f} EUR")
            print(f"Total Storage Cost: {ws.total_storage_cost:.2f} EUR")
            print(f"Inspection Cost: {ws.inspection_cost:.2f} EUR")
            total_cost = ws.total_transport_cost + ws.total_remanufacture_cost + ws.total_storage_cost + ws.inspection_cost
            print(f"Total Cost (Reuse): {total_cost:.2f} EUR")
            total_sale_price = calculate_sale_price(ws)
            print(f"Total Sale Price (Reuse): {total_sale_price:.2f} EUR")
            print(f"Expected Net Revenue (Reuse): {ws.expected_reuse_revenue:.2f} EUR")
            # Sustainability metrics
            print(f"Total Emissions: {ws.total_emissions:.2f} kg CO2")
            print(f"Carbon Cost: {ws.carbon_cost:.2f} EUR")
            print(f"Resource Savings: {ws.resource_savings:.2f} kg of material")
        else:
            print(f"Total Transport Cost (Recycle): {ws.total_transport_cost:.2f} EUR")
            print(f"Inspection Cost: {ws.inspection_cost:.2f} EUR")
            total_recycling_price = calculate_recycling_price(ws)
            print(f"Total Recycling Price: {total_recycling_price:.2f} EUR")
            print(f"Expected Net Revenue (Recycle): {ws.expected_recycle_revenue:.2f} EUR")
            # Sustainability metrics
            print(f"Total Emissions: {ws.total_emissions:.2f} kg CO2")
            print(f"Carbon Cost: {ws.carbon_cost:.2f} EUR")
            print(f"Resource Savings: {ws.resource_savings:.2f} kg of material")

    # Output total profit and sustainability metrics
    print(f"\nOptimal Total Profit: {max_total_profit:.2f} EUR")
    print(f"Total Emissions for All Window Sets: {best_total_emissions:.2f} kg CO2")
    print(f"Total Resource Savings from Reuse: {best_total_resource_savings:.2f} kg of material")

    # Optionally, output warehouse storage and remanufacture facility usage for the optimal combination
    # ...

if __name__ == "__main__":
    main()


Warehouse 1: Location=(73.47, 41.86), Max Storage=900, Daily Cost per Window=1.41 EUR
Warehouse 2: Location=(222.20, 163.61), Max Storage=1000, Daily Cost per Window=3.36 EUR
Remanufacture Facility: Location=(9.53, 28.11), Daily Capacity=64
Recycling Center: Location=(151.61, 7.96)
Reuser (Buyer): Location=(59.65, 194.97), Delivery Date=Day 54

Please enter the number of Window Sets to generate: 10
WindowSet 1: Num Windows=36, Location=(66.13, 176.78), Dimensions=(1.71, 0.51, 0.42), Material=Wood, Condition=Bad, Age=7, Information Complete=True, Demolition Date=Day 8, Delivery Date=Day 37
WindowSet 2: Num Windows=31, Location=(30.66, 113.98), Dimensions=(1.04, 1.02, 0.21), Material=Aluminum, Condition=Bad, Age=30, Information Complete=True, Demolition Date=Day 17, Delivery Date=Day 37
WindowSet 3: Num Windows=34, Location=(23.64, 87.95), Dimensions=(1.44, 1.83, 0.24), Material=Aluminum, Condition=Good, Age=25, Information Complete=True, Demolition Date=Day 1, Delivery Date=Day 37
Windo

# Final version

In [ ]:
import random
import math
from collections import defaultdict
import itertools
import copy

random.seed(42)

# Define Netherlands geographical range (assumed)
NETHERLANDS_X_RANGE = (0, 300)  # x-coordinate range (kilometers)
NETHERLANDS_Y_RANGE = (0, 300)  # y-coordinate range (kilometers)

# Define parameters
TRANSPORT_COST_PER_KM_PER_WINDOW = 0.05   # Transport cost per km per window (EUR)
REMANUFACTURE_COST_PER_WINDOW = 18        # Remanufacture cost per window (EUR)
REUSE_SALE_PRICE_FACTOR = 150             # Base price factor for reuse
RECYCLE_SALE_PRICE_FACTOR = 80            # Base price factor for recycle
REMANUFACTURING_DURATION = 5              # Days required for remanufacturing
DAILY_STORAGE_COST_PER_WINDOW = 2         # Daily storage cost per window (EUR)
INSPECTION_COST_PER_WINDOW = 5            # Inspection cost per window (EUR)

# Sustainability parameters
TRANSPORT_EMISSION_FACTOR = 0.0002        # kg CO2 per km per window
REMANUFACTURING_EMISSION_PER_WINDOW = 5   # kg CO2 per window
RECYCLING_EMISSION_PER_WINDOW = 2         # kg CO2 per window
CARBON_PRICE_PER_KG = 0.05                # EUR per kg CO2

# Define Warehouse class
class Warehouse:
    def __init__(self, name, max_storage):
        self.name = name
        self.x = random.uniform(*NETHERLANDS_X_RANGE)
        self.y = random.uniform(*NETHERLANDS_Y_RANGE)
        self.max_storage = max_storage  # Set max storage based on input
        self.daily_cost_per_window = random.uniform(1, 5)  # Daily storage cost per window (EUR)
        self.current_storage = defaultdict(int)  # Storage per day

    def __str__(self):
        return (f"{self.name}: Location=({self.x:.2f}, {self.y:.2f}), "
                f"Max Storage={self.max_storage}, Daily Cost per Window={self.daily_cost_per_window:.2f} EUR")

# Define Remanufacture Facility class
class RemanufactureFacility:
    def __init__(self):
        self.x = random.uniform(*NETHERLANDS_X_RANGE)
        self.y = random.uniform(*NETHERLANDS_Y_RANGE)
        self.daily_capacity = random.randint(50, 100)  # Daily processing capacity
        self.current_capacity = defaultdict(int)  # Used capacity per day

    def __str__(self):
        return (f"Remanufacture Facility: Location=({self.x:.2f}, {self.y:.2f}), "
                f"Daily Capacity={self.daily_capacity}")

# Define Recycling Center class
class RecyclingCenter:
    def __init__(self):
        self.x = random.uniform(*NETHERLANDS_X_RANGE)
        self.y = random.uniform(*NETHERLANDS_Y_RANGE)

    def __str__(self):
        return f"Recycling Center: Location=({self.x:.2f}, {self.y:.2f})"

# Define Reuser (Buyer) class
class Reuser:
    def __init__(self):
        self.x = random.uniform(*NETHERLANDS_X_RANGE)
        self.y = random.uniform(*NETHERLANDS_Y_RANGE)
        self.delivery_date = random.randint(20, 60)  # Delivery date between day 20 and 60

    def __str__(self):
        return f"Reuser (Buyer): Location=({self.x:.2f}, {self.y:.2f}), Delivery Date=Day {self.delivery_date}"

# Initialize the Reuser before WindowSet class definition
reuser = Reuser()

# Define Window Set class
class WindowSet:
    def __init__(self, set_id):
        self.set_id = set_id
        self.num_windows = random.randint(10, 50)  # Number of windows in the set
        self.x = random.uniform(*NETHERLANDS_X_RANGE)
        self.y = random.uniform(*NETHERLANDS_Y_RANGE)
        self.dimensions = (random.uniform(0.5, 2.0),  # Length (meters)
                           random.uniform(0.5, 2.0),  # Width (meters)
                           random.uniform(0.1, 0.5))  # Height (meters)
        self.material = random.choice(['Material 1', 'Material 2'])  # Material type changed as per instruction
        self.condition = random.choice(['Good', 'Bad'])  # Condition
        self.demolition_date = random.randint(0, 30)  # Demolition date from day 0
        self.delivery_date = reuser.delivery_date     # Use the reuser's delivery date
        self.information_complete = random.choice([True, False])  # Information completeness
        self.age = random.randint(1, 30)  # Age in years
        self.inspection_cost = 0
        self.inspection_done = False
        self.daily_location = {}  # Daily location tracking
        self.total_transport_cost = 0  # Total transport cost
        self.total_remanufacture_cost = 0  # Total remanufacture cost
        self.total_storage_cost = 0       # Total storage cost
        self.status = 'At Original Location'
        self.current_location_x = self.x
        self.current_location_y = self.y
        self.in_warehouse = False
        self.in_remanufacture = False
        self.delivered = False
        self.demolished = False
        self.remanufacture_complete = False
        self.remanufacture_end_day = None
        self.storage_days = 0  # Total days in warehouse
        self.warehouse_name = None  # Name of the warehouse if in warehouse
        self.waiting_for_remanufacture = False  # Waiting in warehouse for remanufacturing
        self.warehouse_entry_day = None  # Day when the window set enters the warehouse
        self.cannot_store = False  # Indicates if the window set cannot be stored
        self.decision = None  # Reuse or Recycle
        self.expected_reuse_revenue = 0
        self.expected_recycle_revenue = 0
        # Sustainability metrics
        self.total_transport_emissions = 0  # kg CO2
        self.total_remanufacture_emissions = 0  # kg CO2
        self.total_recycling_emissions = 0  # kg CO2
        self.resource_savings = 0  # kg of material saved
        self.total_emissions = 0  # Total emissions kg CO2
        self.carbon_cost = 0  # EUR

    def __str__(self):
        return (f"WindowSet {self.set_id}: Num Windows={self.num_windows}, "
                f"Location=({self.x:.2f}, {self.y:.2f}), "
                f"Dimensions=({self.dimensions[0]:.2f}, {self.dimensions[1]:.2f}, {self.dimensions[2]:.2f}), "
                f"Material={self.material}, Condition={self.condition}, "
                f"Age={self.age}, Information Complete={self.information_complete}, "
                f"Demolition Date=Day {self.demolition_date}, Delivery Date=Day {self.delivery_date}")

# Function to calculate distance between two points (kilometers)
def calculate_distance(x1, y1, x2, y2):
    return math.hypot(x2 - x1, y2 - y1)

# Function to find the nearest warehouse with enough capacity
def find_nearest_warehouse_with_capacity(x, y, warehouses, num_windows, start_day, end_day):
    warehouses_with_capacity = []
    for wh in warehouses:
        capacity_available = True
        for day in range(start_day, end_day):
            current_storage = wh.current_storage.get(day, 0)
            if current_storage + num_windows > wh.max_storage:
                capacity_available = False
                break
        if capacity_available:
            distance = calculate_distance(x, y, wh.x, wh.y)
            warehouses_with_capacity.append((distance, wh))
    if warehouses_with_capacity:
        warehouses_with_capacity.sort(key=lambda x: x[0])
        return warehouses_with_capacity[0][1], warehouses_with_capacity[0][0]
    else:
        return None, None  # No warehouse with enough capacity

# Function to calculate sale price based on window characteristics
def calculate_sale_price(ws):
    area = ws.dimensions[0] * ws.dimensions[1]  # Window area in square meters
    material_factor = 1.2 if ws.material == 'Material 1' else 1.0  # Adjusted for new materials
    price_per_window = area * material_factor * REUSE_SALE_PRICE_FACTOR
    total_price = price_per_window * ws.num_windows
    return total_price

# Function to calculate recycling price based on window characteristics
def calculate_recycling_price(ws):
    area = ws.dimensions[0] * ws.dimensions[1]  # Window area in square meters
    # Random material factor for recycling, adjusted for new materials
    material_factor = random.uniform(0.5, 1.0) if ws.material == 'Material 1' else random.uniform(0.3, 0.8)
    price_per_window = area * material_factor * RECYCLE_SALE_PRICE_FACTOR
    total_price = price_per_window * ws.num_windows
    return total_price

# Function to estimate window weight (kg) based on material and dimensions
def estimate_window_weight(ws):
    # Adjust densities according to new materials
    density = 2700 if ws.material == 'Material 1' else 600  # kg/m^3
    volume = ws.dimensions[0] * ws.dimensions[1] * ws.dimensions[2]  # m^3
    weight = density * volume  # kg per window
    total_weight = weight * ws.num_windows
    return total_weight

# Function to simulate the process for a given combination of decisions
def simulate_process(window_sets, decisions, warehouses, remanufacture_facility, recycling_center, reuser, total_days=60):
    # Reset facilities
    for warehouse in warehouses:
        warehouse.current_storage = defaultdict(int)
    remanufacture_facility.current_capacity = defaultdict(int)

    total_profit = 0
    total_emissions = 0  # Total emissions for all window sets (kg CO2)

    # For each window set, assign the decision
    for ws, decision in zip(window_sets, decisions):
        ws.decision = decision
        # Reset window set state
        ws.daily_location = {}
        ws.total_transport_cost = 0
        ws.total_remanufacture_cost = 0
        ws.total_storage_cost = 0
        ws.inspection_cost = 0
        ws.inspection_done = False
        ws.status = 'At Original Location'
        ws.current_location_x = ws.x
        ws.current_location_y = ws.y
        ws.in_warehouse = False
        ws.in_remanufacture = False
        ws.delivered = False
        ws.demolished = False
        ws.remanufacture_complete = False
        ws.remanufacture_end_day = None
        ws.storage_days = 0
        ws.warehouse_name = None
        ws.waiting_for_remanufacture = False
        ws.warehouse_entry_day = None
        ws.cannot_store = False
        # Reset sustainability metrics
        ws.total_transport_emissions = 0
        ws.total_remanufacture_emissions = 0
        ws.total_recycling_emissions = 0
        ws.resource_savings = 0
        ws.total_emissions = 0
        ws.carbon_cost = 0

    # Adjust decisions based on age
    for ws in window_sets:
        if ws.age > 25:
            ws.decision = 'Recycle'

    # Simulate day by day
    for day in range(total_days):
        # Initialize daily capacities and storage if not already (handled by defaultdict)

        for ws in window_sets:
            if day < ws.demolition_date:
                ws.daily_location[day] = 'At Original Location'
                continue

            if not ws.inspection_done and not ws.information_complete:
                # Undergo inspection
                ws.daily_location[day] = 'Under Inspection'
                ws.inspection_cost = INSPECTION_COST_PER_WINDOW * ws.num_windows
                ws.inspection_done = True
                continue  # Move to next day

            if not ws.demolished:
                ws.demolished = True
                # Demolition day actions
                if ws.decision == 'Recycle':
                    # Transport to recycling center
                    distance = calculate_distance(ws.current_location_x, ws.current_location_y,
                                                  recycling_center.x, recycling_center.y)
                    transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                    ws.total_transport_cost += transport_cost
                    # Calculate transport emissions
                    transport_emissions = distance * TRANSPORT_EMISSION_FACTOR * ws.num_windows
                    ws.total_transport_emissions += transport_emissions
                    ws.current_location_x, ws.current_location_y = recycling_center.x, recycling_center.y
                    ws.daily_location[day] = 'Sent to Recycling Center'
                    # Recycling emissions
                    recycling_emissions = RECYCLING_EMISSION_PER_WINDOW * ws.num_windows
                    ws.total_recycling_emissions += recycling_emissions
                    # Total emissions
                    ws.total_emissions = ws.total_transport_emissions + ws.total_recycling_emissions
                    ws.delivered = True  # Considered as processed
                    continue  # No further action needed
                else:
                    # Reuse decision
                    if ws.condition == 'Bad':
                        # Try to process in remanufacture facility
                        can_process = True
                        for d in range(day, day + REMANUFACTURING_DURATION):
                            if remanufacture_facility.current_capacity[d] + ws.num_windows > remanufacture_facility.daily_capacity:
                                can_process = False
                                break

                        if can_process:
                            # Schedule remanufacturing and update capacity
                            for d in range(day, day + REMANUFACTURING_DURATION):
                                remanufacture_facility.current_capacity[d] += ws.num_windows
                            # Transport to remanufacture facility
                            distance = calculate_distance(ws.current_location_x, ws.current_location_y,
                                                          remanufacture_facility.x, remanufacture_facility.y)
                            transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                            ws.total_transport_cost += transport_cost
                            # Calculate transport emissions
                            transport_emissions = distance * TRANSPORT_EMISSION_FACTOR * ws.num_windows
                            ws.total_transport_emissions += transport_emissions
                            ws.current_location_x, ws.current_location_y = remanufacture_facility.x, remanufacture_facility.y
                            ws.daily_location[day] = 'In Remanufacture Facility'
                            ws.in_remanufacture = True
                            ws.remanufacture_end_day = day + REMANUFACTURING_DURATION - 1
                            # Remanufacture cost
                            remanufacture_cost = REMANUFACTURE_COST_PER_WINDOW * ws.num_windows
                            ws.total_remanufacture_cost += remanufacture_cost
                            # Remanufacturing emissions
                            remanufacture_emissions = REMANUFACTURING_EMISSION_PER_WINDOW * ws.num_windows
                            ws.total_remanufacture_emissions += remanufacture_emissions
                        else:
                            # Try to find a warehouse with enough capacity
                            # Assume maximum storage duration until total_days
                            storage_end_day = total_days
                            nearest_warehouse, distance = find_nearest_warehouse_with_capacity(
                                ws.current_location_x, ws.current_location_y, warehouses, ws.num_windows, day, storage_end_day)
                            if nearest_warehouse is not None:
                                # Transport to warehouse
                                transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                                ws.total_transport_cost += transport_cost
                                # Calculate transport emissions
                                transport_emissions = distance * TRANSPORT_EMISSION_FACTOR * ws.num_windows
                                ws.total_transport_emissions += transport_emissions
                                ws.current_location_x, ws.current_location_y = nearest_warehouse.x, nearest_warehouse.y
                                ws.daily_location[day] = f"In {nearest_warehouse.name}"
                                ws.in_warehouse = True
                                ws.warehouse_name = nearest_warehouse.name
                                ws.waiting_for_remanufacture = True  # Waiting for remanufacturing
                                ws.warehouse_entry_day = day
                                # Update warehouse storage for the expected storage period
                                for d in range(day, storage_end_day):
                                    nearest_warehouse.current_storage[d] += ws.num_windows
                            else:
                                # No warehouse has enough capacity; cannot proceed
                                ws.daily_location[day] = 'Cannot be stored in warehouse'
                                ws.cannot_store = True
                                continue  # Cannot proceed further
                    else:
                        # Good condition
                        if day < ws.delivery_date:
                            # Estimate storage duration until delivery date
                            storage_end_day = ws.delivery_date
                            nearest_warehouse, distance = find_nearest_warehouse_with_capacity(
                                ws.current_location_x, ws.current_location_y, warehouses, ws.num_windows, day, storage_end_day)
                            if nearest_warehouse is not None:
                                # Transport to warehouse
                                transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                                ws.total_transport_cost += transport_cost
                                # Calculate transport emissions
                                transport_emissions = distance * TRANSPORT_EMISSION_FACTOR * ws.num_windows
                                ws.total_transport_emissions += transport_emissions
                                ws.current_location_x, ws.current_location_y = nearest_warehouse.x, nearest_warehouse.y
                                ws.daily_location[day] = f"In {nearest_warehouse.name}"
                                ws.in_warehouse = True
                                ws.warehouse_name = nearest_warehouse.name
                                ws.warehouse_entry_day = day
                                # Update warehouse storage for the expected storage period
                                for d in range(day, storage_end_day):
                                    nearest_warehouse.current_storage[d] += ws.num_windows
                            else:
                                # No warehouse has enough capacity
                                ws.daily_location[day] = 'Cannot be stored in warehouse'
                                ws.cannot_store = True
                                continue  # Cannot proceed further
                        else:
                            # Transport to reuser
                            distance = calculate_distance(ws.current_location_x, ws.current_location_y, reuser.x, reuser.y)
                            transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                            ws.total_transport_cost += transport_cost
                            # Calculate transport emissions
                            transport_emissions = distance * TRANSPORT_EMISSION_FACTOR * ws.num_windows
                            ws.total_transport_emissions += transport_emissions
                            ws.current_location_x, ws.current_location_y = reuser.x, reuser.y
                            ws.daily_location[day] = 'Delivered to Reuser'
                            ws.delivered = True
                            ws.in_warehouse = False
                            # Resource savings
                            total_weight = estimate_window_weight(ws)
                            ws.resource_savings = total_weight  # kg of material saved by reusing
                            # Total emissions (transport emissions already accounted)
                            ws.total_emissions = ws.total_transport_emissions
                            continue  # Delivered, no further action needed
                    continue  # Move to next day

            if ws.decision == 'Recycle':
                # Already processed
                continue

            if ws.in_remanufacture:
                if day <= ws.remanufacture_end_day:
                    ws.daily_location[day] = 'In Remanufacture Facility'
                    # Capacity already accounted for
                else:
                    ws.in_remanufacture = False
                    ws.remanufacture_complete = True
                    # After remanufacturing
                    if day < ws.delivery_date:
                        # Estimate storage duration until delivery date
                        storage_end_day = ws.delivery_date
                        nearest_warehouse, distance = find_nearest_warehouse_with_capacity(
                            ws.current_location_x, ws.current_location_y, warehouses, ws.num_windows, day, storage_end_day)
                        if nearest_warehouse is not None:
                            # Transport to warehouse
                            transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                            ws.total_transport_cost += transport_cost
                            # Calculate transport emissions
                            transport_emissions = distance * TRANSPORT_EMISSION_FACTOR * ws.num_windows
                            ws.total_transport_emissions += transport_emissions
                            ws.current_location_x, ws.current_location_y = nearest_warehouse.x, nearest_warehouse.y
                            ws.daily_location[day] = f"In {nearest_warehouse.name}"
                            ws.in_warehouse = True
                            ws.warehouse_name = nearest_warehouse.name
                            ws.warehouse_entry_day = day
                            # Update warehouse storage for the expected storage period
                            for d in range(day, storage_end_day):
                                nearest_warehouse.current_storage[d] += ws.num_windows
                        else:
                            # No warehouse has enough capacity
                            ws.daily_location[day] = 'Cannot be stored in warehouse'
                            ws.cannot_store = True
                            continue  # Cannot proceed further
                    else:
                        # Transport to reuser
                        distance = calculate_distance(ws.current_location_x, ws.current_location_y, reuser.x, reuser.y)
                        transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                        ws.total_transport_cost += transport_cost
                        # Calculate transport emissions
                        transport_emissions = distance * TRANSPORT_EMISSION_FACTOR * ws.num_windows
                        ws.total_transport_emissions += transport_emissions
                        ws.current_location_x, ws.current_location_y = reuser.x, reuser.y
                        ws.daily_location[day] = 'Delivered to Reuser'
                        ws.delivered = True
                        ws.in_warehouse = False
                        # Resource savings
                        total_weight = estimate_window_weight(ws)
                        ws.resource_savings = total_weight  # kg of material saved by reusing
                        # Total emissions (transport and remanufacturing emissions already accounted)
                        ws.total_emissions = ws.total_transport_emissions + ws.total_remanufacture_emissions
                        continue  # Delivered, no further action needed
                continue  # Move to next day

            if ws.in_warehouse:
                ws.daily_location[day] = f"In {ws.warehouse_name}"
                # Accumulate storage cost
                storage_cost = DAILY_STORAGE_COST_PER_WINDOW * ws.num_windows
                ws.total_storage_cost += storage_cost

                if ws.waiting_for_remanufacture:
                    # Try to move to remanufacture facility
                    can_process = True
                    for d in range(day, day + REMANUFACTURING_DURATION):
                        if remanufacture_facility.current_capacity[d] + ws.num_windows > remanufacture_facility.daily_capacity:
                            can_process = False
                            break
                    if can_process:
                        # Remove from warehouse and update storage
                        ws.in_warehouse = False
                        ws.waiting_for_remanufacture = False
                        warehouse_exit_day = day
                        storage_end_day = total_days  # For waiting_for_remanufacture, storage_end_day is total_days
                        # Update warehouse storage by removing windows from current day to storage_end_day
                        for d in range(warehouse_exit_day, storage_end_day):
                            for warehouse in warehouses:
                                if warehouse.name == ws.warehouse_name:
                                    warehouse.current_storage[d] -= ws.num_windows
                                    if warehouse.current_storage[d] < 0:
                                        warehouse.current_storage[d] = 0  # Prevent negative storage
                                    break
                        ws.warehouse_name = None
                        # Schedule remanufacturing and update capacity
                        for d in range(day, day + REMANUFACTURING_DURATION):
                            remanufacture_facility.current_capacity[d] += ws.num_windows
                        # Transport to remanufacture facility
                        distance = calculate_distance(ws.current_location_x, ws.current_location_y,
                                                      remanufacture_facility.x, remanufacture_facility.y)
                        transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                        ws.total_transport_cost += transport_cost
                        # Calculate transport emissions
                        transport_emissions = distance * TRANSPORT_EMISSION_FACTOR * ws.num_windows
                        ws.total_transport_emissions += transport_emissions
                        ws.current_location_x, ws.current_location_y = remanufacture_facility.x, remanufacture_facility.y
                        ws.daily_location[day] = 'In Remanufacture Facility'
                        ws.in_remanufacture = True
                        ws.remanufacture_end_day = day + REMANUFACTURING_DURATION - 1
                        # Remanufacture cost
                        remanufacture_cost = REMANUFACTURE_COST_PER_WINDOW * ws.num_windows
                        ws.total_remanufacture_cost += remanufacture_cost
                        # Remanufacturing emissions
                        remanufacture_emissions = REMANUFACTURING_EMISSION_PER_WINDOW * ws.num_windows
                        ws.total_remanufacture_emissions += remanufacture_emissions
                        continue  # Move to next day

                if day == ws.delivery_date and not ws.waiting_for_remanufacture:
                    # Transport to reuser
                    distance = calculate_distance(ws.current_location_x, ws.current_location_y, reuser.x, reuser.y)
                    transport_cost = distance * TRANSPORT_COST_PER_KM_PER_WINDOW * ws.num_windows
                    ws.total_transport_cost += transport_cost
                    # Calculate transport emissions
                    transport_emissions = distance * TRANSPORT_EMISSION_FACTOR * ws.num_windows
                    ws.total_transport_emissions += transport_emissions
                    ws.current_location_x, ws.current_location_y = reuser.x, reuser.y
                    ws.daily_location[day] = 'Delivered to Reuser'
                    ws.delivered = True
                    ws.in_warehouse = False
                    # Update warehouse storage by removing windows from current day to storage_end_day
                    warehouse_exit_day = day
                    storage_end_day = ws.delivery_date
                    for d in range(warehouse_exit_day, storage_end_day):
                        for warehouse in warehouses:
                            if warehouse.name == ws.warehouse_name:
                                warehouse.current_storage[d] -= ws.num_windows
                                if warehouse.current_storage[d] < 0:
                                    warehouse.current_storage[d] = 0  # Prevent negative storage
                                break
                    ws.warehouse_name = None
                    # Resource savings
                    total_weight = estimate_window_weight(ws)
                    ws.resource_savings = total_weight  # kg of material saved by reusing
                    # Total emissions (transport emissions already accounted)
                    ws.total_emissions = ws.total_transport_emissions
                    continue  # Delivered, no further action needed
                continue  # Move to next day

        # End of day loop

    # After simulation, calculate revenues and costs
    total_profit = 0
    total_emissions = 0  # kg CO2
    total_resource_savings = 0  # kg of material saved

    for ws in window_sets:
        if ws.decision == 'Reuse' and not ws.cannot_store:
            total_sale_price = calculate_sale_price(ws)
            total_cost = ws.total_transport_cost + ws.total_remanufacture_cost + ws.total_storage_cost + ws.inspection_cost
            ws.expected_reuse_revenue = total_sale_price - total_cost
            # Carbon cost
            ws.carbon_cost = ws.total_emissions * CARBON_PRICE_PER_KG
            ws.expected_reuse_revenue -= ws.carbon_cost
            total_profit += ws.expected_reuse_revenue
            total_emissions += ws.total_emissions
            total_resource_savings += ws.resource_savings
        elif ws.decision == 'Recycle':
            total_recycling_price = calculate_recycling_price(ws)
            # Transport cost to recycling center (already calculated)
            total_cost = ws.total_transport_cost + ws.inspection_cost
            ws.expected_recycle_revenue = total_recycling_price - total_cost
            # Carbon cost
            ws.carbon_cost = ws.total_emissions * CARBON_PRICE_PER_KG
            ws.expected_recycle_revenue -= ws.carbon_cost
            total_profit += ws.expected_recycle_revenue
            total_emissions += ws.total_emissions
            # No resource savings in recycling
        else:
            # Could not proceed with reuse
            total_cost = ws.total_transport_cost + ws.total_remanufacture_cost + ws.total_storage_cost + ws.inspection_cost
            ws.expected_reuse_revenue = - total_cost
            # Carbon cost
            ws.carbon_cost = ws.total_emissions * CARBON_PRICE_PER_KG
            ws.expected_reuse_revenue -= ws.carbon_cost
            total_profit += ws.expected_reuse_revenue  # This would be negative
            total_emissions += ws.total_emissions
            # No resource savings
    return total_profit, total_emissions, total_resource_savings

# Main execution
def main():
    # Initialize facilities
    warehouse1 = Warehouse("Warehouse 1", 900)
    warehouse2 = Warehouse("Warehouse 2", 1000)
    warehouses = [warehouse1, warehouse2]
    remanufacture_facility = RemanufactureFacility()
    recycling_center = RecyclingCenter()
    reuser = Reuser()

    # Output facility information
    print(warehouse1)
    print(warehouse2)
    print(remanufacture_facility)
    print(recycling_center)
    print(reuser)

    # Ask user for the number of window sets to generate
    num_window_sets = int(input("\nPlease enter the number of Window Sets to generate: "))

    # Generate window sets
    window_sets = []
    for i in range(1, num_window_sets + 1):
        ws = WindowSet(i)
        window_sets.append(ws)
        print(ws)

    # Generate decision options per window set
    decision_options_list = []
    for ws in window_sets:
        if ws.age > 25:
            decision_options_list.append(['Recycle'])
        else:
            decision_options_list.append(['Reuse', 'Recycle'])

    # Generate all possible combinations of decisions
    all_combinations = list(itertools.product(*decision_options_list))

    max_total_profit = float('-inf')
    min_total_profit = float('inf')

    best_combination = None
    best_window_sets = None
    best_total_emissions = None
    best_total_resource_savings = None

    print(f"\nTotal combinations to evaluate: {len(all_combinations)}")

    # For progress tracking (optional)
    combination_count = 0

    for decisions in all_combinations:
        combination_count += 1
        # Deep copy the window sets and facilities to avoid interference
        window_sets_copy = copy.deepcopy(window_sets)
        warehouses_copy = [copy.deepcopy(warehouse1), copy.deepcopy(warehouse2)]
        remanufacture_facility_copy = copy.deepcopy(remanufacture_facility)

        total_profit, total_emissions, total_resource_savings = simulate_process(
            window_sets_copy, decisions, warehouses_copy, remanufacture_facility_copy, recycling_center, reuser)

        if total_profit > max_total_profit:
            max_total_profit = total_profit
            best_combination = decisions
            best_window_sets = window_sets_copy
            best_total_emissions = total_emissions
            best_total_resource_savings = total_resource_savings

        # Optional: Print progress
        # print(f"Evaluated combination {combination_count}/{len(all_combinations)}: Total Profit = {total_profit:.2f} EUR")

    # Output the results for the best combination
    print(f"\nOptimal Total Profit: {max_total_profit:.2f} EUR")
    print(f"Total Emissions: {best_total_emissions:.2f} kg CO2")
    print(f"Total Resource Savings: {best_total_resource_savings:.2f} kg of material")
    print(f"Best Combination of Decisions: {best_combination}")



    for decisions in all_combinations:
        combination_count += 1
        # Deep copy the window sets and facilities to avoid interference
        window_sets_copy = copy.deepcopy(window_sets)
        warehouses_copy = [copy.deepcopy(warehouse1), copy.deepcopy(warehouse2)]
        remanufacture_facility_copy = copy.deepcopy(remanufacture_facility)

        total_profit, total_emissions, total_resource_savings = simulate_process(
            window_sets_copy, decisions, warehouses_copy, remanufacture_facility_copy, recycling_center, reuser)

        if total_profit < min_total_profit:
            min_total_profit = total_profit
            worst_combination = decisions
            worst_window_sets = window_sets_copy
            worst_total_emissions = total_emissions
            worst_total_resource_savings = total_resource_savings

        # Optional: Print progress
        # print(f"Evaluated combination {combination_count}/{len(all_combinations)}: Total Profit = {total_profit:.2f} EUR")

    # Output the results for the worst combination
    print(f"\nMinimal Total Profit: {min_total_profit:.2f} EUR")
    print(f"Total Emissions: {worst_total_emissions:.2f} kg CO2")
    print(f"Total Resource Savings: {worst_total_resource_savings:.2f} kg of material")
    print(f"Worst Combination of Decisions: {worst_combination}")





    # For each window set, display the information
    for ws in best_window_sets:
        print(f"\nWindow Set {ws.set_id} Decision: {ws.decision}")
        # Output daily locations in the specified format
        print(f"Daily Locations for WindowSet {ws.set_id}:")
        last_location = None
        start_day = None
        days = sorted(ws.daily_location.keys())
        for day in days:
            location = ws.daily_location[day]
            if location != last_location:
                if last_location is not None:
                    print(f"Day {start_day} - Day {day - 1}: {last_location}")
                start_day = day
                last_location = location
        # Print the last period
        if last_location is not None:
            print(f"Day {start_day} - Day {day}: {last_location}")

        # Output estimated costs and revenues
        print(f"\nEstimated Costs and Revenues for WindowSet {ws.set_id}:")
        if ws.decision == 'Reuse':
            print(f"Total Transport Cost (Reuse): {ws.total_transport_cost:.2f} EUR")
            print(f"Total Remanufacture Cost: {ws.total_remanufacture_cost:.2f} EUR")
            print(f"Total Storage Cost: {ws.total_storage_cost:.2f} EUR")
            print(f"Inspection Cost: {ws.inspection_cost:.2f} EUR")
            total_cost = ws.total_transport_cost + ws.total_remanufacture_cost + ws.total_storage_cost + ws.inspection_cost
            print(f"Total Cost (Reuse): {total_cost:.2f} EUR")
            total_sale_price = calculate_sale_price(ws)
            print(f"Total Sale Price (Reuse): {total_sale_price:.2f} EUR")
            print(f"Carbon Cost: {ws.carbon_cost:.2f} EUR")
            print(f"Expected Net Revenue (Reuse): {ws.expected_reuse_revenue:.2f} EUR")
            # Sustainability metrics
            print(f"Total Emissions: {ws.total_emissions:.2f} kg CO2")
            print(f"Resource Savings: {ws.resource_savings:.2f} kg of material")
        else:
            print(f"Total Transport Cost (Recycle): {ws.total_transport_cost:.2f} EUR")
            print(f"Inspection Cost: {ws.inspection_cost:.2f} EUR")
            total_recycling_price = calculate_recycling_price(ws)
            print(f"Total Recycling Price: {total_recycling_price:.2f} EUR")
            print(f"Carbon Cost: {ws.carbon_cost:.2f} EUR")
            print(f"Expected Net Revenue (Recycle): {ws.expected_recycle_revenue:.2f} EUR")
            # Sustainability metrics
            print(f"Total Emissions: {ws.total_emissions:.2f} kg CO2")
            print(f"Resource Savings: {ws.resource_savings:.2f} kg of material")

    # Output total profit and sustainability metrics
    print(f"\nOptimal Total Profit: {max_total_profit:.2f} EUR")
    print(f"Total Emissions for All Window Sets: {best_total_emissions:.2f} kg CO2")
    print(f"Total Resource Savings from Reuse: {best_total_resource_savings:.2f} kg of material")

    # Optionally, output warehouse storage and remanufacture facility usage for the optimal combination
    # ...

if __name__ == "__main__":
    main()


Warehouse 1: Location=(73.47, 41.86), Max Storage=900, Daily Cost per Window=1.41 EUR
Warehouse 2: Location=(222.20, 163.61), Max Storage=1000, Daily Cost per Window=3.36 EUR
Remanufacture Facility: Location=(9.53, 28.11), Daily Capacity=64
Recycling Center: Location=(151.61, 7.96)
Reuser (Buyer): Location=(59.65, 194.97), Delivery Date=Day 54

Please enter the number of Window Sets to generate: 10
WindowSet 1: Num Windows=36, Location=(66.13, 176.78), Dimensions=(1.71, 0.51, 0.42), Material=Material 2, Condition=Bad, Age=7, Information Complete=True, Demolition Date=Day 8, Delivery Date=Day 37
WindowSet 2: Num Windows=31, Location=(30.66, 113.98), Dimensions=(1.04, 1.02, 0.21), Material=Material 1, Condition=Bad, Age=30, Information Complete=True, Demolition Date=Day 17, Delivery Date=Day 37
WindowSet 3: Num Windows=34, Location=(23.64, 87.95), Dimensions=(1.44, 1.83, 0.24), Material=Material 1, Condition=Good, Age=25, Information Complete=True, Demolition Date=Day 1, Delivery Date=Da

# End